# Lab 5: Microsoft Agent Framework (MAF) - Workflow Patterns and Orchestration

## Overview

In this notebook, we combine **Azure AI Foundry Agent** and **Microsoft Agent Framework (MAF)** to practice various workflow patterns and multi-agent orchestration.

### Key Concepts

**🤖 AI Agents (Individual Agents) - Using Azure AI Foundry Agent**
- Defining agents using the Agent API from Azure AI Foundry
- Processing user input with LLM
- Performing tasks via tool and MCP server calls
- State management based on threads
- Supporting multi-turn conversations

**⚡ Workflows - Using Microsoft Agent Framework (MAF)**
- Defining complex orchestrations with MAF’s graph-based workflows
- Combining multiple agents and functions defined with Foundry Agent
- Type safety and state management
- Conditional routing, parallel processing, dynamic execution

### Architecture: Foundry Agent + MAF Workflow

```
┌──────────────────────────────────────────────────────────────────┐
│              MAF Workflow Orchestration Layer                    │
│                                                                  │
│  ┌────────────────┐  ┌────────────────┐  ┌────────────────┐   │
│  │  Sequential    │  │  Concurrent    │  │  Handoff       │   │
│  │  Workflow      │  │  Workflow      │  │  Workflow      │   │
│  └───────┬────────┘  └───────┬────────┘  └───────┬────────┘   │
│          │                   │                    │            │
└──────────┼───────────────────┼────────────────────┼────────────┘
           │                   │                    │
┌──────────▼───────────────────▼────────────────────▼────────────┐
│           Azure AI Foundry Agents (Agent Layer)                │
│                                                                 │
│  ┌─────────────────┐  ┌─────────────────┐  ┌────────────────┐ │
│  │  Validator      │  │  Transformer    │  │  Summarizer    │ │
│  │  Agent          │  │  Agent          │  │  Agent         │ │
│  │  (Foundry)      │  │  (Foundry)      │  │  (Foundry)     │ │
│  └─────────────────┘  └─────────────────┘  └────────────────┘ │
│                                                                 │
│  ✅ Thread-based State Management                              │
│  ✅ LLM Integration (GPT-4o)                      │
│  ✅ Tool/MCP Server Integration                                │
└─────────────────────────────────────────────────────────────────┘
```

### Orchestration Patterns to Learn

Workflow patterns to implement using MAF:

1. **Sequential Pattern (Lab 2)** - Sequential Execution
   - Passing the result of one Foundry Agent to the next agent
   - Suitable for pipelines, multi-stage processing

2. **Concurrent Pattern (Lab 3)** - Simultaneous Execution (Parallel)
   - Broadcasting tasks to all agents at the same time
   - Suitable for parallel analysis, ensemble decision-making

3. **Conditional Pattern (Lab 4)** - Conditional Routing
   - Routing to different specialized agents based on input conditions
   - Suitable for intent classification, personalized responses

4. **Loop Pattern (Lab 5)** - Iterative Improvement
   - Executing a feedback loop until conditions are satisfied
   - Suitable for quality enhancement, iterative optimization

5. **Error Handling Pattern (Lab 6)** - Error Detection and Recovery
   - Detecting failures, providing alternatives, and automated recovery
   - Suitable for reservation systems requiring stability

6. **Handoff Pattern (Lab 7)** - Dynamic Control Transfer
   - Dynamically transferring control between agents based on complexity
   - Suitable for escalation and expert assignment

### Prerequisites

- ✅ Python 3.10+
- ✅ Azure AI Foundry Project setup
- ✅ Microsoft Agent Framework (MAF) installation
- ✅ Azure OpenAI or OpenAI API setup
- ✅ Basic understanding of Python asyncio

---

## 📑 Table of Contents

### Initial Setup
1. **Essential Libraries and Environment Setup**
2. **Initialization of Azure AI Foundry Agent Client**

### Workflow Patterns Lab
2. **Sequential Pattern** - Sequential Execution Pattern
3. **Concurrent Pattern** - Parallel Execution Pattern  
4. **Conditional Pattern** - Conditional Branching Pattern
5. **Loop Pattern** - Iterative Improvement Pattern
6. **Error Handling Pattern** - Error Handling Pattern
7. **Handoff Pattern** - Dynamic Routing Pattern

### Summary
- **Comprehensive Workflow Pattern Comparison**
- **Key Concepts from Each Lab**
- **Best Practices and Optimization**
- **Production Guide**

---

---

## ⚙️ Before You Start

**Select Python Kernel:**

1. Click **"Select Kernel"** at the top right of the notebook 
2. Choose **"Python Environments..."**
3. Select **`.venv (Python 3.x.x)`** (the virtual environment created in the project root)

> 💡 **GitHub Codespaces**: In Codespaces, the `.venv` environment is automatically created.  
> If `.venv` is not visible, create it via the terminal using `python -m venv .venv`.

---

## 1. Required Libraries and Environment Setup (Import & Setup)

In [ ]:
# ========================================================================
# 📦 Required Libraries Import
# ========================================================================

import os
import sys
import time
import asyncio
import logging
from datetime import datetime
from typing import Dict, Any, List, Optional
from dataclasses import dataclass

# Azure AI Foundry
from azure.ai.projects import AIProjectClient
from azure.identity import AzureCliCredential, ChainedTokenCredential, ManagedIdentityCredential
from azure.identity.aio import (
    AzureCliCredential as AsyncAzureCliCredential,
    ManagedIdentityCredential as AsyncManagedIdentityCredential,
    ChainedTokenCredential as AsyncChainedTokenCredential
)

# Microsoft Agent Framework (MAF) - WorkflowBuilder
from agent_framework.azure import AzureAIAgentClient
from agent_framework import WorkflowBuilder, WorkflowContext, executor

# Logging settings - concise output
logging.basicConfig(
    level=logging.WARNING,
    format='%(levelname)s: %(message)s'
)
logger = logging.getLogger(__name__)

# Adjust Azure SDK log level
logging.getLogger('azure').setLevel(logging.ERROR)
logging.getLogger('azure.core.pipeline.policies.http_logging_policy').setLevel(logging.ERROR)
logging.getLogger('azure.identity').setLevel(logging.ERROR)

print("✅ Required libraries imported successfully")
print(f"   - Python version: {sys.version.split()[0]}")
print(f"   - Azure AI Projects SDK loaded successfully")
print(f"   - Microsoft Agent Framework (MAF) WorkflowBuilder loaded successfully")
print(f"   - Asyncio ready")


In [ ]:
# ========================================================================
# 🔐 Verify Azure CLI Login
# ========================================================================

import subprocess
import json

def check_azure_login():
    """Verify Azure CLI login status and return detailed information"""
    try:
        result = subprocess.run(
            ["az", "account", "show"],
            capture_output=True,
            text=True,
            timeout=10
        )
        if result.returncode == 0:
            account_info = json.loads(result.stdout)
            return True, account_info
        return False, None
    except Exception as e:
        return False, str(e)

print("\n" + "="*70)
print("🔐 Verifying Azure CLI Authentication Status")
print("="*70)

is_logged_in, account_info = check_azure_login()

if is_logged_in:
    print("\n✅ Azure CLI Authentication Successful\n")
    print(f"📋 Account Information:")
    print(f"   ├─ Subscription Name: {account_info.get('name', 'N/A')}")
    print(f"   ├─ Subscription ID: {account_info.get('id', 'N/A')}")
    print(f"   ├─ Tenant ID: {account_info.get('tenantId', 'N/A')}")
    print(f"   ├─ User: {account_info.get('user', {}).get('name', 'N/A')}")
    print(f"   └─ State: {account_info.get('state', 'N/A')}")
    
    print(f"\n🎯 Ready to use Foundry Agent")
    print(f"   Access to Azure AI Project resources is available.\n")
else:
    print("\n❌ Azure CLI Authentication Failed\n")
    print(f"⚠️  Issue:")
    print(f"   {account_info if account_info else 'Not logged in'}\n")
    print(f"🔧 Resolution:")
    print(f"   1. Execute the following command in the terminal:")
    print(f"      $ az login")
    print(f"   2. Complete authentication in the browser")
    print(f"   3. Run this cell again\n")

print("="*70 + "\n")


## 🔧 Azure AI Foundry Client Initialization

This section details the initialization of the client required for communication with the **Azure AI Foundry Agent**.

### Initialization Process:

**1️⃣ Loading Configuration File**
- Read project connection string and model information from the `config.json` file.
- Configuration can also be set via environment variables (environment variables take precedence).

**2️⃣ Parsing Connection String**
- Extract the Azure AI Project endpoint URL.
- Connection string format: `https://...;subscription_id=...;resource_group=...`
- Use only the portion before the semicolon (`;`) as the endpoint.

**3️⃣ Authentication Setup**
- Utilize `AzureCliCredential` for authentication (based on Azure CLI login).
- Prior authentication required using the `az login` command.

**4️⃣ Creating AIProjectClient**
- Create the client using the endpoint and credentials.
- This will serve as the base client for calling the Foundry Agent API.

**5️⃣ Verification**
- Check if the connection is successful.
- Provide troubleshooting methods in case of errors.

### Prerequisites:
- ✅ Azure CLI installed and logged in (`az login`)
- ✅ `config.json` file with the `project_connection_string` set
- ✅ Azure AI Project resource created and with access permissions

### Configuration Details:
- **Connection String**: Azure AI Project endpoint URL
- **Model Deployment**: Name of the LLM model to be used (e.g., `gpt-4o`)

In [ ]:
# ========================================================================
# 🔧 MAF Agent Client Initialization
# ========================================================================

print("\n" + "="*70)
print("🔧 Initializing Microsoft Agent Framework (MAF) Client")
print("="*70)

# Load settings from config.json
import json
config_path = "config.json"
project_endpoint = None
model_name = None  # Load from config.json

if os.path.exists(config_path):
    with open(config_path, 'r') as f:
        config = json.load(f)
        project_connection_string = config.get("project_connection_string")
        
        # Extract endpoint only from the connection string (part before the first ;)
        if project_connection_string and ';' in project_connection_string:
            project_endpoint = project_connection_string.split(';')[0]
        else:
            project_endpoint = project_connection_string
        
        # Load model_deployment_name from config.json
        model_name = config.get("model_deployment_name")
        
        if not model_name:
            print(f"\n⚠️  'model_deployment_name' setting is missing in config.json")
            print(f"   Default value 'gpt-4o' will be used")
            model_name = "gpt-4o"
        
        print(f"\n✅ Successfully loaded config.json")
else:
    print(f"\n⚠️  Unable to find the config.json file")
    print(f"   Default value 'gpt-4o' will be used")
    model_name = "gpt-4o"

# Settings can also be configured via environment variables (Priority: Environment Variables > config.json)
if os.getenv("AZURE_AI_PROJECT_ENDPOINT"):
    project_endpoint = os.getenv("AZURE_AI_PROJECT_ENDPOINT")

if os.getenv("AZURE_AI_MODEL_DEPLOYMENT_NAME"):
    model_name = os.getenv("AZURE_AI_MODEL_DEPLOYMENT_NAME")

print(f"\n📋 Configuration details:")
print(f"   ├─ Project Endpoint: {project_endpoint[:50] + '.' if project_endpoint else '❌ Not configured'}")
print(f"   └─ Model Deployment: {model_name}")

# Initialize MAF Agent Client
agent_client = None

if not project_endpoint:
    print(f"\n⚠️  Project endpoint is not configured")
    print(f"   Please check the config.json file or set the environment variable.")
    print(f"   Example: export AZURE_AI_PROJECT_ENDPOINT='<your-endpoint-url>'")
else:
    try:
        # Use Azure CLI authentication (Async credential required)
        async_credential = AsyncChainedTokenCredential(
            AsyncManagedIdentityCredential(),
            AsyncAzureCliCredential()
        )
        
        # Create MAF Agent Client
        agent_client = AzureAIAgentClient(
            project_endpoint=project_endpoint,
            model_deployment_name=model_name,
            async_credential=async_credential
        )
        
        print(f"\n✅ Successfully initialized MAF Agent Client")
        print(f"   - Authentication: Azure CLI / Managed Identity (Async)")
        print(f"   - Endpoint: {project_endpoint}")
        print(f"   - Model: {model_name}")
        print(f"   - WorkflowBuilder is available")
        
    except Exception as e:
        print(f"\n❌ Failed to initialize MAF Agent Client")
        print(f"   Error: {type(e).__name__}: {str(e)}")
        print(f"\n💡 Suggestions for troubleshooting:")
        print(f"   1. Run az login")
        print(f"   2. Verify the config.json file")
        print(f"   3. Check the Azure AI Project resources")
        print(f"   4. Verify agent_framework package installation: pip install agent-framework[azure-ai]")
        agent_client = None

print("="*70 + "\n")


## Workflow Pattern Concepts

### 1️⃣ Sequential Pattern
```
Input → Task 1 → Task 2 → Task 3 → Output
        (after completion)  (after completion)
```
- **Features**: Each task is executed sequentially, the result of the previous task is used by the next task
- **Use Cases**: Data transformation pipeline, multi-step validation
- **Advantages**: Intuitive, simple to implement
- **Disadvantages**: Slow execution (parallelization not possible)

### 2️⃣ Parallel Pattern
```
      ┌─ Task 1 ┐
Input ┤─ Task 2 ├─ Aggregation → Output
      └─ Task 3 ┘
```
- **Features**: Executing multiple tasks concurrently
- **Use Cases**: Independent information gathering, multi-agent queries
- **Advantages**: Fast execution, efficient resource utilization
- **Disadvantages**: Requires synchronization management

### 3️⃣ Conditional Branching Pattern
```
       ┌─ (Condition A) → Path 1
Input ─┤─ (Condition B) → Path 2
       └─ (Default value) → Path 3
            ↓
         Output
```
- **Features**: Execution follows different paths based on conditions
- **Use Cases**: User intent classification, dynamic routing
- **Advantages**: Flexible control
- **Disadvantages**: Increased complexity

### 4️⃣ Loop Pattern
```
Input → [Check loop condition]
         ├─ (Continue) → Task → Increment loop counter
         └─ (Exit) → Output
```
- **Features**: Repeated execution until the condition is satisfied
- **Use Cases**: Batch processing, data refinement
- **Advantages**: Dynamic iteration possible
- **Disadvantages**: Beware of infinite loops

### 5️⃣ Handoff Pattern
```
Input → Agent A → [Handoff condition]
                    ├─ (Expert needed) → Agent B (Specialist)
                    ├─ (Approval needed) → Agent C (Approver)
                    └─ (Can be completed) → Output
```
- **Features**: Dynamic control transfer to different agents based on context or conditions
- **Use Cases**: Customer support escalation, approval workflows, expert consultations
- **Advantages**: Flexible control transition, utilization of expertise
- **Disadvantages**: Requires management of context transfer between agents

### 6️⃣ Error Handling & Retry Pattern
```
Input → Try Task → [Error occurred?]
                   ├─ (Yes) → Retry (exp backoff) → Max attempts?
                   │          ├─ (Not reached) → Retry
                   │          └─ (Reached) → Fallback
                   └─ (No) → Output
```
- **Features**: Retry and fallback strategies in case of errors
- **Use Cases**: API calls, network operations
- **Advantages**: Stability improvement
- **Disadvantages**: Implementation complexity

## 🛠️ MAF WorkflowBuilder Utility

This section defines the essential components for using the **WorkflowBuilder pattern** of MAF.

### MAF WorkflowBuilder Overview

**WorkflowBuilder** is the core orchestration tool in MAF:

**Key Components:**
- ✅ **@executor decorator**: Defines each node (step) of the workflow
- ✅ **WorkflowContext**: Manages data transfer and state between nodes
- ✅ **WorkflowBuilder**: Constructs the execution graph by connecting nodes and edges
- ✅ **Agent**: Executes actual LLM calls within each node

**Execution Flow:**
```
1. Create Agent → 2. Define Executor Node → 3. Build Graph with WorkflowBuilder
                   ↓
4. Execute Workflow → 5. Transfer Data via Context → 6. Collect Results
```

### Message Type Definition

Define the message types to be used in the workflow as a `dataclass`.

In [ ]:
# ========================================================================
# 📦 MAF Message Types (For Workflow Data Transfer)
# ========================================================================

@dataclass
class TravelRequest:
    """Travel request message"""
    destination: str
    days: int
    user_query: str
    itinerary: Optional[str] = None
    local_insights: Optional[str] = None
    final_plan: Optional[str] = None

# ========================================================================
# 🤖 Create Agents (Experts for Each Step)
# ========================================================================

def create_travel_agents():
    """Create 3 expert agents for travel planning"""
    
    if not agent_client:
        raise ValueError("❌ MAF Agent client has not been initialized.")
    
    print("\n" + "="*70)
    print("🤖 Creating travel expert agents.")
    print("="*70)
    
    # Agent 1: Travel Planner (Draft the basic itinerary)
    travel_planner = agent_client.create_agent(
        name="TravelPlanner",
        instructions=(
            "You are a professional travel planner.\n\n"
            "Role:\n"
            "- Analyze the user's travel request to create a detailed day-by-day itinerary\n"
            "- Suggest key attractions and activities for each day\n"
            "- Include detailed plans by time period (morning/lunch/evening)\n"
            "- Provide descriptions of each location and noteworthy sights\n"
            "- Offer guidance on travel times and transportation methods\n\n"
            "Style: Specific and actionable itinerary"
        )
    )
    
    # Agent 2: Local Expert (Add local insights)
    local_expert = agent_client.create_agent(
        name="LocalExpert",
        instructions=(
            "You are a local travel expert.\n\n"
            "Role:\n"
            "- Review the existing travel itinerary and add unique places known only to locals\n"
            "- Recommend hidden gems that tourists may not know\n"
            "- Introduce diners, cafes, or restaurants frequented by locals\n"
            "- Suggest opportunities for special local cultural experiences\n"
            "- Provide practical tips for each location\n\n"
            "Style: Authentic and intriguing local experiences"
        )
    )
    
    # Agent 3: Travel Summarizer (Final integrated plan)
    travel_summarizer = agent_client.create_agent(
        name="TravelSummarizer",
        instructions=(
            "You are a travel plan summarization expert.\n\n"
            "Role:\n"
            "- Compile all suggestions and advice into a complete final travel plan\n"
            "- Consistently integrate detailed daily schedules\n"
            "- Include practical travel tips (transportation, language, etiquette)\n"
            "- Add cautions and safety information\n"
            "- Provide budget guidelines\n"
            "- Summarize essential reservation details\n\n"
            "Style: Detailed, comprehensive, and practical itinerary"
        )
    )
    
    print(f"✅ Agent 1: {travel_planner.name} (Travel Planner) successfully created")
    print(f"✅ Agent 2: {local_expert.name} (Local Expert) successfully created")
    print(f"✅ Agent 3: {travel_summarizer.name} (Travel Summarizer) successfully created")
    print("="*70 + "\n")
    
    return travel_planner, local_expert, travel_summarizer

# Create Agents
travel_planner, local_expert, travel_summarizer = create_travel_agents()


## 2. Sequential Workflow Pattern (Sequential Workflow)

### 🔧 Implementation of Sequential Pattern using MAF WorkflowBuilder

In [ ]:
"""
Sequential Pattern - Using MAF WorkflowBuilder
- Define each step (node) with the @executor decorator
- Connect nodes and construct the execution graph with WorkflowBuilder
- Deliver data between steps using WorkflowContext
- Call LLM at each step using Azure AI Foundry Agent
"""

# ========================================================================
# Step 1: Define Executor Nodes (Define each step using @executor)
# ========================================================================

@executor(id="travel_planner")
async def travel_planner_node(msg: TravelRequest, ctx: WorkflowContext[TravelRequest]) -> None:
    """Step 1: Travel Planner - Create a basic itinerary"""
    print(f"\n🗺️  Step 1: Travel Planner - Creating a basic itinerary.")
    print(f"   Input: {msg.user_query}")
    
    # Generate a basic itinerary with the Travel Planner Agent
    query = f"""Create a detailed schedule for the following travel request:

Request: {msg.user_query}

Please create a daily schedule that includes:
- Major attractions and activities for each day
- Detailed plans for breakfast, lunch, and dinner
- Characteristics and highlights of each place
- Travel time and transportation methods
- Recommended restaurants and local cuisine

Create a specific and actionable itinerary."""
    
    # Execute the agent (create a new thread)
    thread = travel_planner.get_new_thread()
    result = await travel_planner.run(query, thread=thread)
    response = result.text if hasattr(result, 'text') else str(result)
    
    # Save result in Context and pass it to the next step
    msg.itinerary = response
    await ctx.send_message(msg, target_id="local_expert")
    print(f"✅ Step 1 Complete: Basic itinerary creation complete")


@executor(id="local_expert")
async def local_expert_node(msg: TravelRequest, ctx: WorkflowContext[TravelRequest]) -> None:
    """Step 2: Local Expert - Add local insights"""
    print(f"\n🏛️  Step 2: Local Expert - Adding local insights.")
    
    # Add local insights using the Local Expert Agent
    query = f"""Review the following travel itinerary and add special places or activities known by locals:

Existing itinerary:
{msg.itinerary}

Add the following:
- Hidden spots unknown to typical tourists
- Restaurants or cafes frequented by locals
- Unique opportunities for cultural experiences
- Practical tips for each location

Make the schedule richer and more authentic."""
    
    # Execute the agent
    thread = local_expert.get_new_thread()
    result = await local_expert.run(query, thread=thread)
    response = result.text if hasattr(result, 'text') else str(result)
    
    # Save result in Context and pass it to the next step
    msg.local_insights = response
    await ctx.send_message(msg, target_id="travel_summarizer")
    print(f"✅ Step 2 Complete: Local insights added")


@executor(id="travel_summarizer")
async def travel_summarizer_node(msg: TravelRequest, ctx: WorkflowContext[TravelRequest]) -> None:
    """Step 3: Travel Summarizer - Create a final integrated plan"""
    print(f"\n📋 Step 3: Travel Summarizer - Creating the final plan.")
    
    # Integrate the final plan using the Travel Summarizer Agent
    query = f"""Integrate all the following information to create a complete final travel plan:

Original Request: {msg.user_query}

Basic itinerary:
{msg.itinerary}

Local insights:
{msg.local_insights}

Include the following in the final plan:
1. Fully integrated detailed daily itinerary
2. Practical travel tips (transportation, language, etiquette)
3. Precautions and safety information
4. Budget guidelines
5. Essential reservation information

Create a consistent and fully integrated final travel plan."""
    
    # Execute the agent
    thread = travel_summarizer.get_new_thread()
    result = await travel_summarizer.run(query, thread=thread)
    response = result.text if hasattr(result, 'text') else str(result)
    
    # Save the final result in Context and output it
    msg.final_plan = response
    await ctx.yield_output(msg)
    print(f"✅ Step 3 Complete: Final plan creation complete")


# ========================================================================
# Step 2: Connect Nodes and Construct Execution Graph with WorkflowBuilder
# ========================================================================

sequential_workflow = (
    WorkflowBuilder()
    .set_start_executor(travel_planner_node)          # Set the start node
    .add_edge(travel_planner_node, local_expert_node)  # Step 1 → Step 2
    .add_edge(local_expert_node, travel_summarizer_node)  # Step 2 → Step 3
    .build()
)

print("\n" + "="*70)
print("✅ Sequential Workflow graph construction complete")
print("="*70)
print("📊 Workflow structure:")
print("   travel_planner_node (Step 1)")
print("         ↓")
print("   local_expert_node (Step 2)")
print("         ↓")
print("   travel_summarizer_node (Step 3)")
print("="*70 + "\n")


# ========================================================================
# Step 3: Execute Workflow
# ========================================================================

async def run_sequential_workflow():
    """Function to execute the Sequential Workflow"""
    print("\n" + "="*70)
    print("🚀 Beginning Sequential Workflow execution (MAF WorkflowBuilder)")
    print("="*70)

    # Create input message
    user_query = "Plan a 5-day trip to London."
    travel_request = TravelRequest(
        destination="London",
        days=5,
        user_query=user_query
    )

    # Execute workflow (run_stream - async generator)
    outputs = []
    async for event in sequential_workflow.run_stream(travel_request):
        # Extract output from the event
        if hasattr(event, 'output') and event.output is not None:
            outputs.append(event.output)
            print(f"📤 Event received: {type(event.output).__name__}")
        elif hasattr(event, 'data') and event.data is not None:
            outputs.append(event.data)
            print(f"📤 Event received: {type(event.data).__name__}")

    # Extract the final result (last output is the final TravelRequest)
    if outputs:
        final_result = outputs[-1]
    else:
        raise ValueError("❌ No output received from the workflow.")

    print("\n" + "="*70)
    print("📊 Sequential Pattern Result (MAF WorkflowBuilder)")
    print("="*70)
    print(f"\n📝 Travel Request: {final_result.user_query}\n")
    print(f"\n{'='*70}")
    print(f"🗺️  Step 1 - Travel Planner (Basic Itinerary):")
    print(f"{'='*70}")
    print(f"{final_result.itinerary}\n")
    print(f"\n{'='*70}")
    print(f"🏛️  Step 2 - Local Expert (Local Insights):")
    print(f"{'='*70}")
    print(f"{final_result.local_insights}\n")
    print(f"\n{'='*70}")
    print(f"📋 Step 3 - Travel Summarizer (Final Integrated Plan):")
    print(f"{'='*70}")
    print(f"{final_result.final_plan}")
    print(f"\n{'='*70}")
    
    return final_result

# Execute workflow
final_result = await run_sequential_workflow()

**Sequential Pattern Analysis (Using MAF WorkflowBuilder)**

### ✅ Key Components

**1. Message Type (Data Transmission)**
- Define the data structure used throughout the workflow with the `TravelRequest` dataclass
- Each node modifies and passes the same message object

**2. Executor Nodes (@executor decorator)**
- `@executor(id="travel_planner")`: Step 1 - Create a basic itinerary
- `@executor(id="local_expert")`: Step 2 - Add local insights  
- `@executor(id="travel_summarizer")`: Step 3 - Integrate the final plan

**3. WorkflowBuilder (Construct Execution Graph)**
```python
workflow = (
    WorkflowBuilder()
    .set_start_executor(travel_planner_node)           # Start point
    .add_edge(travel_planner_node, local_expert_node)  # Step 1 → 2
    .add_edge(local_expert_node, travel_summarizer_node)  # Step 2 → 3
    .build()
)
```

**4. WorkflowContext (Control Flow Between Nodes)**
- `await ctx.send_message(msg, next_node)`: Handoff control to the next node
- `await ctx.yield_output(msg)`: Output the final result

### ✅ Execution Flow

```
TravelRequest creation
    ↓
travel_planner_node (Foundry Agent: travel_planner)
    → msg.itinerary is created
    → ctx.ctx.send_message(msg, local_expert_node)
    ↓
local_expert_node (Foundry Agent: local_expert)
    → msg.local_insights is created
    → ctx.ctx.send_message(msg, travel_summarizer_node)
    ↓
travel_summarizer_node (Foundry Agent: travel_summarizer)
    → msg.final_plan is created
    → ctx.yield_output(msg)
    ↓
Final results collected
```

### ✅ Advantages of MAF WorkflowBuilder

1. **Clear Execution Graph**: Easily visualize node connection structure
2. **Type Safety**: Clearly define message structures using dataclasses
3. **Flexible Control Flow**: Enable dynamic routing with handoff_to
4. **State Management**: WorkflowContext tracks execution states
5. **Reusability**: Each executor node can be reused across different workflows

### ✅ Use Cases
- Multistage data transformation pipelines
- Document creation workflows (Draft → Review → Final Version)
- Complex request handling (Analysis → Planning → Execution)
- Tasks requiring iterative improvement

## 3. Concurrent Workflow Pattern (Concurrent Execution Workflow)

In [ ]:
"""
Concurrent Pattern - Using MAF WorkflowBuilder
- Define each expert node with the @executor decorator
- Construct parallel execution graph with WorkflowBuilder
- Transfer data via WorkflowContext
- Multiple experts analyze independently and simultaneously
"""

# ========================================================================
# Definition of Message Types (For Concurrent Pattern)
# ========================================================================

@dataclass
class TravelAnalysisRequest:
    """Travel analysis request message (for parallel processing)"""
    destination: str
    user_query: str
    culture_history: Optional[str] = None
    food_dining: Optional[str] = None
    practical_tips: Optional[str] = None
    final_summary: Optional[str] = None

# ========================================================================
# Creation of Concurrent Pattern Agents
# ========================================================================

def create_concurrent_agents():
    """Create 3 expert agents for parallel analysis"""
    
    if not agent_client:
        raise ValueError("❌ MAF Agent client has not been initialized.")
    
    print("\n" + "="*70)
    print("🤖 Creating parallel analysis expert agents.")
    print("="*70)
    
    # Agent 1: Culture & History Expert
    culture_expert = agent_client.create_agent(
        name="CultureHistoryExpert",
        instructions=(
            "You are an expert in culture and history.\n\n"
            "Role:\n"
            "- Thorough analysis of cultural and historical values of the travel destination\n"
            "- Recommend must-visit historical sites (museums, heritage sites, memorials)\n"
            "- Explain the historical significance and background of each location\n"
            "- Suggest cultural experience activities (traditional performances, festivals, workshops)\n"
            "- Provide information on local etiquette and cultural considerations\n"
            "- Recommend visit times and tips\n\n"
            "Style: Focused on in-depth and educational cultural experiences"
        )
    )
    
    # Agent 2: Food & Dining Expert
    food_expert = agent_client.create_agent(
        name="FoodDiningExpert",
        instructions=(
            "You are a cuisine expert.\n\n"
            "Role:\n"
            "- Introduce the food culture and culinary experiences of the travel destination\n"
            "- Recommend must-try local foods and cuisines\n"
            "- Suggest a variety of restaurants (Michelin-starred, local favorites, street food)\n"
            "- Explain the characteristics and origins of each dish\n"
            "- Provide budgeting guides for meals (low/mid/high budget)\n"
            "- Offer information for vegetarians/allergy accommodations\n"
            "- Indicate reservation requirements and operating hours\n\n"
            "Style: Focused on authentic culinary experiences"
        )
    )
    
    # Agent 3: Practical Travel Expert
    practical_expert = agent_client.create_agent(
        name="PracticalTravelExpert",
        instructions=(
            "You are an expert in practical travel information.\n\n"
            "Role:\n"
            "- Provide essential practical information for travelers\n"
            "- Transportation info (airport-to-city transfer, public transport, transport cards)\n"
            "- Accommodation info (recommended areas for stays, price ranges, features)\n"
            "- Budget guide (daily budget: accommodation/transport/food/tourism)\n"
            "- Communication info (SIM cards, WiFi, recommended apps)\n"
            "- Safety tips and emergency contact information\n"
            "- Shopping info (recommended areas, duty-free shops, souvenirs)\n"
            "- Weather and clothing guides\n\n"
            "Style: Focused on practical and useful information"
        )
    )
    
    # Agent 4: Final Summarizer (Integrates results from parallel analysis)
    final_summarizer = agent_client.create_agent(
        name="FinalSummarizer",
        instructions=(
            "You are an expert in comprehensive travel guide creation.\n\n"
            "Role:\n"
            "- Integrate analysis results from multiple experts into one comprehensive guide\n"
            "- Organize cultural/history, culinary, and practical information cohesively\n"
            "- Create a complete guide that travelers can use directly\n"
            "- Highlight key points of each domain\n"
            "- Propose daily itineraries\n"
            "- Provide overall travel tips and considerations\n\n"
            "Style: Comprehensive and practical travel guide"
        )
    )
    
    print(f"✅ Agent 1: {culture_expert.name} (Culture & History Expert) successfully created")
    print(f"✅ Agent 2: {food_expert.name} (Food & Dining Expert) successfully created")
    print(f"✅ Agent 3: {practical_expert.name} (Practical Travel Expert) successfully created")
    print(f"✅ Agent 4: {final_summarizer.name} (Final Summarizer) successfully created")
    print("="*70 + "\n")
    
    return culture_expert, food_expert, practical_expert, final_summarizer

# Create Concurrent Agents
culture_expert, food_expert, practical_expert, final_summarizer = create_concurrent_agents()

# ========================================================================
# Step 1: Define Executor Nodes (Expert nodes to be executed concurrently)
# ========================================================================

@executor(id="broadcast_start")
async def broadcast_start_node(msg: TravelAnalysisRequest, ctx: WorkflowContext[TravelAnalysisRequest]) -> None:
    """Start Node: Broadcast to all parallel nodes"""
    print(f"\n📡 Broadcast Start: Distributing tasks to all experts.")
    # Send message to three parallel nodes simultaneously
    await ctx.send_message(msg, target_id="culture_history_expert")
    await ctx.send_message(msg, target_id="food_dining_expert")
    await ctx.send_message(msg, target_id="practical_tips_expert")
    print(f"✅ Broadcast completed: Simultaneously starting 3 expert analyses")


@executor(id="culture_history_expert")
async def culture_history_node(msg: TravelAnalysisRequest, ctx: WorkflowContext[TravelAnalysisRequest]) -> None:
    """Concurrent Step 1: Culture & History Expert - Culture/History Analysis"""
    print(f"\n🏛️  Parallel Node 1: Culture & History Expert - Analyzing.")
    
    query = f"""{msg.user_query} Provide recommendations from a cultural and historical perspective:

Include the following:
- Must-visit historical sites (museums, heritage sites, memorials)
- Historical significance and background of these locations
- Cultural experience activities (traditional performances, festivals, workshops)
- Local etiquette and cultural considerations
- Recommended visit times and tips

Provide in-depth cultural experiences"""
    
    # Execute Agent
    thread = culture_expert.get_new_thread()
    result = await culture_expert.run(query, thread=thread)
    response = result.text if hasattr(result, 'text') else str(result)
    
    # Save results in Context and send to aggregator
    msg.culture_history = response
    await ctx.send_message(msg, target_id="aggregator")
    print(f"✅ Parallel Node 1 completed: Culture/History analysis finished")


@executor(id="food_dining_expert")
async def food_dining_node(msg: TravelAnalysisRequest, ctx: WorkflowContext[TravelAnalysisRequest]) -> None:
    """Concurrent Step 2: Food & Dining Expert - Food/Culinary Analysis"""
    print(f"\n🍽️  Parallel Node 2: Food & Dining Expert - Analyzing.")
    
    query = f"""{msg.user_query} Provide recommendations from a food and culinary perspective:

Include the following:
- Must-try local foods and cuisines
- Recommended restaurants (Michelin-starred, local favorites, street food)
- Characteristics and origins of each dish
- Budget guides for meals (low/mid/high budget)
- Information for vegetarians/allergy accommodations
- Reservation requirements and operating hours

Provide an authentic culinary experience"""
    
    # Execute Agent
    thread = food_expert.get_new_thread()
    result = await food_expert.run(query, thread=thread)
    response = result.text if hasattr(result, 'text') else str(result)
    
    # Save results in Context and send to aggregator
    msg.food_dining = response
    await ctx.send_message(msg, target_id="aggregator")
    print(f"✅ Parallel Node 2 completed: Food/Culinary analysis finished")


@executor(id="practical_tips_expert")
async def practical_tips_node(msg: TravelAnalysisRequest, ctx: WorkflowContext[TravelAnalysisRequest]) -> None:
    """Concurrent Step 3: Practical Travel Expert - Practical Information Analysis"""
    print(f"\n🧳 Parallel Node 3: Practical Travel Expert - Analyzing.")
    
    query = f"""{msg.user_query} Provide practical travel information:

Include the following:
- Transportation: airport-to-city transfer, public transport, transport cards
- Accommodation: recommended stays by area (location, price range, features)
- Budget: daily budget guide (accommodation/transport/food/tourism)
- Communication: SIM cards, WiFi, recommended apps
- Safety: safety guidelines, emergency contacts
- Shopping: recommended shopping areas, duty-free stores, souvenirs
- Weather: seasonal clothing guidance, umbrellas/sunscreen, etc.

Provide practical and useful information"""
    
    # Execute Agent
    thread = practical_expert.get_new_thread()
    result = await practical_expert.run(query, thread=thread)
    response = result.text if hasattr(result, 'text') else str(result)
    
    # Save results in Context and send to aggregator
    msg.practical_tips = response
    await ctx.send_message(msg, target_id="aggregator")
    print(f"✅ Parallel Node 3 completed: Practical Information Analysis finished")


@executor(id="aggregator")
async def aggregator_node(msg: TravelAnalysisRequest, ctx: WorkflowContext[TravelAnalysisRequest]) -> None:
    """Aggregation Step: Combine all parallel analysis results"""
    print(f"\n📊 Aggregator: Combining analysis results from all experts.")
    
    # Combine results from all parallel analyses and create the final summary
    query = f"""Integrate the analysis results from multiple experts to create a complete travel guide:

Original query: {msg.user_query}

=== Culture/History Expert Analysis ===
{msg.culture_history}

=== Food/Culinary Expert Analysis ===
{msg.food_dining}

=== Practical Information Expert Analysis ===
{msg.practical_tips}

Using the above information, provide:
1. A comprehensive travel guide
2. Daily itinerary recommendations (combining suggestions from all fields)
3. Highlight key points from each domain
4. General travel tips and considerations
5. A practical format that travelers can use immediately

Write a thorough and cohesive final travel guide"""
    
    # Execute Final Summarizer Agent
    thread = final_summarizer.get_new_thread()
    result = await final_summarizer.run(query, thread=thread)
    response = result.text if hasattr(result, 'text') else str(result)
    
    # Save the final result in Context and output
    msg.final_summary = response
    await ctx.yield_output(msg)
    print(f"✅ Aggregator completed: Final comprehensive guide created")


# ========================================================================
# Step 2: Build Parallel Execution Graph with WorkflowBuilder
# ========================================================================

concurrent_workflow = (
    WorkflowBuilder()
    # Start from Broadcast Node
    .set_start_executor(broadcast_start_node)
    # Broadcast → Fan-out to 3 parallel nodes
    .add_edge(broadcast_start_node, culture_history_node)
    .add_edge(broadcast_start_node, food_dining_node)
    .add_edge(broadcast_start_node, practical_tips_node)
    # Fan-in from 3 parallel nodes → aggregator
    .add_edge(culture_history_node, aggregator_node)
    .add_edge(food_dining_node, aggregator_node)
    .add_edge(practical_tips_node, aggregator_node)
    .build()
)

print("\n" + "="*70)
print("✅ Concurrent Workflow graph construction completed")
print("="*70)
print("📊 Workflow structure (Concurrent Pattern):")
print("   broadcast_start_node (Start)")
print("         ↓")
print("   ┌─────┼─────┐")
print("   ↓     ↓     ↓")
print("   culture  food  practical")
print("   history  dining  tips")
print("   (Concurrent1)  (Concurrent2)  (Concurrent3)")
print("   └─────┼─────┘")
print("         ↓")
print("   aggregator_node (Integration)")
print("="*70 + "\n")


# ========================================================================
# Step 3: Execute Workflow
# ========================================================================

async def run_concurrent_workflow():
    """Function to execute the Concurrent Workflow"""
    print("\n" + "="*70)
    print("🚀 Concurrent Workflow execution starting (MAF WorkflowBuilder)")
    print("="*70)

    # Create input message
    user_query = "Plan a 5-day trip to London."
    analysis_request = TravelAnalysisRequest(
        destination="London",
        user_query=user_query
    )

    # Execute the workflow (use run_stream - async generator)
    outputs = []
    async for event in concurrent_workflow.run_stream(analysis_request):
        # Extract output from events
        if hasattr(event, 'output') and event.output is not None:
            outputs.append(event.output)
            print(f"📤 Event received: {type(event.output).__name__}")
        elif hasattr(event, 'data') and event.data is not None:
            outputs.append(event.data)
            print(f"📤 Event received: {type(event.data).__name__}")

    # Extract the final result
    if outputs:
        final_result = outputs[-1]
    else:
        raise ValueError("❌ No output received from the workflow.")

    print("\n" + "="*70)
    print("📊 Concurrent Pattern Result (MAF WorkflowBuilder)")
    print("="*70)
    print(f"\n📝 Travel Request: {final_result.user_query}\n")
    print(f"\n{'='*70}")
    print(f"🏛️  Parallel Analysis 1 - Culture & History Expert:")
    print(f"{'='*70}")
    print(f"{final_result.culture_history}\n")
    print(f"\n{'='*70}")
    print(f"🍽️  Parallel Analysis 2 - Food & Dining Expert:")
    print(f"{'='*70}")
    print(f"{final_result.food_dining}\n")
    print(f"\n{'='*70}")
    print(f"🧳 Parallel Analysis 3 - Practical Travel Expert:")
    print(f"{'='*70}")
    print(f"{final_result.practical_tips}\n")
    print(f"\n{'='*70}")
    print(f"📋 Final Integrated Guide - Final Summarizer:")
    print(f"{'='*70}")
    print(f"{final_result.final_summary}")
    print(f"\n{'='*70}")
    
    return final_result

# Execute Workflow
concurrent_result = await run_concurrent_workflow()


**Concurrent Pattern Analysis (Using MAF WorkflowBuilder)**

### ✅ Key Components

**1. Message Type (Data Transmission)**
- Define the data structure to store parallel analysis results with the `TravelAnalysisRequest` dataclass.
- Each expert node independently saves its analysis results in the message object.

**2. Executor Nodes (@executor Decorators)**
- `@executor(id="culture_history_expert")`: Parallel Node 1 - Culture/History Analysis
- `@executor(id="food_dining_expert")`: Parallel Node 2 - Food/Dining Analysis
- `@executor(id="practical_tips_expert")`: Parallel Node 3 - Practical Information Analysis
- `@executor(id="aggregator")`: Aggregator Node - Collect all results and summarize.

**3. WorkflowBuilder (Constructing Parallel Execution Graphs)**
```python
workflow = (
    WorkflowBuilder()
    # Start three nodes concurrently (parallel execution)
    .set_start_executor(culture_history_node)
    .set_start_executor(food_dining_node)
    .set_start_executor(practical_tips_node)
    # Converge all parallel nodes → aggregator
    .add_edge(culture_history_node, aggregator_node)
    .add_edge(food_dining_node, aggregator_node)
    .add_edge(practical_tips_node, aggregator_node)
    .build()
)
```

**4. WorkflowContext (Collecting and Integrating Results)**
- Each parallel node executes independently and saves results to a message.
- The Aggregator node waits for the completion of all parallel results and integrates them.
- `await ctx.yield_output(msg)`: Outputs the final integrated results.

### ✅ Execution Flow

```
Create TravelAnalysisRequest
    ↓
┌────────────────┬────────────────┬────────────────┐
│ culture_history│  food_dining   │ practical_tips │
│     _node      │     _node      │     _node      │
│  Foundry Agent │ Foundry Agent  │ Foundry Agent  │
└────────┬───────┴────────┬───────┴────────┬───────┘
         │                │                │
         └────────────────┼────────────────┘
                          ↓
                  aggregator_node
                  (Integrate all results)
                  Foundry Agent
                          ↓
                  Final Integrated Guide
```

### ✅ Advantages of MAF WorkflowBuilder Parallel Pattern

1. **True Parallel Execution**: Simultaneous start with multiple `set_start_executor()` calls.
2. **Automatic Synchronization**: Aggregator node automatically waits for all parallel nodes to complete.
3. **Type Safety**: Clearly defines each expert's results with a dataclass.
4. **Scalability**: Easy to add new expert nodes.
5. **Performance Improvement**: Reduces execution time by approximately 66% compared to sequential processing (for three nodes).

### ✅ Differences from Sequential Pattern

| Feature          | Sequential Pattern     | Concurrent Pattern      |
|-------------------|-----------------------|-------------------------|
| **Node Start**    | Single `.set_start_executor()` call | Multiple `.set_start_executor()` calls |
| **Execution**     | Sequential (1→2→3)    | Parallel (1, 2, 3 simultaneously) |
| **Data Flow**     | Result of previous → Next node | Independent for each node → Aggregator integration |
| **Use Case**      | Step-by-step improvement | Independent analysis required |
| **Execution Time**| Sum (T1+T2+T3)        | Maximum (max(T1, T2, T3)) |

### ✅ Use Cases
- Comprehensive analysis from diverse perspectives (culture, food, practical tips, etc.)
- Autonomous data collection tasks (parallel API calls).
- Ensemble decision-making (independent assessments from multiple models).
- Multi-agent brainstorming:
  - When independent information gathering is required.
  - Rapid response for multi-domain queries.

**Architecture Summary**:
```
MAF Concurrent Workflow (Parallel Expert Analysis)
         ↓
    asyncio.gather()
    ↙      ↓      ↘
Expert 1  Expert 2  Expert 3
(Culture) (Food)    (Practical)
     ↘      ↓      ↙
       Result Integration
```

**Real-Life Example**:
- Input: "Create a 3-day trip plan for Paris"
- Expert 1: Recommends historical and cultural attractions like the Louvre and Eiffel Tower.
- Expert 2: Suggests Michelin-star restaurants, bistros, and cafes.
- Expert 3: Provides metro passes, accommodation areas, and budgeting guides.
- Result: Comprehensive travel information from three perspectives.

## 4. Conditional Branching Pattern (Conditional Branching Workflow)

In [ ]:
"""
Conditional Branching Pattern - Using MAF WorkflowBuilder
- Define each conditional node with the @executor decorator
- Build conditional branching graph using WorkflowBuilder
- Dynamic routing to different experts based on travel style
"""

# ========================================================================
# Define Message Type (For Conditional Branching Pattern)
# ========================================================================

@dataclass
class TravelStyleRequest:
    """Request message by travel style"""
    user_query: str
    travel_style: Optional[str] = None
    specialized_plan: Optional[str] = None

# ========================================================================
# Create Conditional Pattern Agents
# ========================================================================

def create_conditional_agents():
    """Create expert agents for conditional branching"""
    
    if not agent_client:
        raise ValueError("❌ MAF Agent client has not been initialized.")
    
    print("\n" + "="*70)
    print("🤖 Creating conditional branching expert agents.")
    print("="*70)
    
    # Agent 1: Style Classifier (Travel Style Classification)
    style_classifier = agent_client.create_agent(
        name="StyleClassifier",
        instructions=(
            "You are an expert in classifying travel styles.\n\n"
            "Role:\n"
            "- Analyze the user's travel request to identify the primary travel style\n"
            "- Precisely classify it into one of the following:\n"
            "  * Culture: Focused on culture/history (museums, historical sites, traditions)\n"
            "  * Activity: Focused on activities (sports, adventure, outdoor activities)\n"
            "  * Relaxation: Focused on leisure (resorts, spas, relaxation)\n"
            "  * Gourmet: Focused on gastronomy (food tours, cooking experiences, wineries)\n\n"
            "Important: Respond with only one word among 'Culture', 'Activity', 'Relaxation', or 'Gourmet'\n"
            "Style: Concise and precise classification"
        )
    )
    
    # Agent 2: Culture Expert (Culture/History Expert)
    culture_expert = agent_client.create_agent(
        name="CultureExpert",
        instructions=(
            "You are an expert in culture/history travel.\n\n"
            "Role:\n"
            "- Plan trips centered around museums, art galleries, and historical landmarks\n"
            "- Explain the historical significance and background of each place\n"
            "- Propose traditional cultural experiences\n"
            "- Include recommendations for at least 5 places and activities\n"
            "- Provide visiting times, durations, and budget guides\n\n"
            "Style: Focused on profound cultural experiences"
        )
    )
    
    # Agent 3: Activity Expert (Activity Expert)
    activity_expert = agent_client.create_agent(
        name="ActivityExpert",
        instructions=(
            "You are an expert in activity-focused travel.\n\n"
            "Role:\n"
            "- Plan trips around outdoor activities, adventure sports, and hiking\n"
            "- Include schedules for water sports, mountain climbing, cycling, and other active pursuits\n"
            "- Provide guidance on each activity's difficulty and necessary equipment\n"
            "- Include at least 5 recommended activities and locations\n"
            "- Provide safety rules and booking information\n\n"
            "Style: Focused on dynamic experiences"
        )
    )
    
    # Agent 4: Relaxation Expert (Relaxation Expert)
    relaxation_expert = agent_client.create_agent(
        name="RelaxationExpert",
        instructions=(
            "You are an expert in relaxation travel.\n\n"
            "Role:\n"
            "- Plan trips centered around resorts, spas, and wellness programs\n"
            "- Recommend tranquil places and scenic natural spots\n"
            "- Propose wellness programs, meditation, and yoga experiences\n"
            "- Include recommendations for at least 5 relaxation spots and programs\n"
            "- Provide accommodation details and amenities information\n\n"
            "Style: Focused on peaceful and rejuvenating experiences"
        )
    )
    
    # Agent 5: Gourmet Expert (Gourmet Expert)
    gourmet_expert = agent_client.create_agent(
        name="GourmetExpert",
        instructions=(
            "You are an expert in gastronomy-focused travel.\n\n"
            "Role:\n"
            "- Plan trips centered around Michelin restaurants and food tours\n"
            "- Include cooking experiences, winery visits, and food market explorations\n"
            "- Describe unique features and signature menus of each restaurant\n"
            "- Include recommendations for at least 5 restaurants and gastronomic activities\n"
            "- Provide reservation methods, dress codes, and price ranges\n\n"
            "Style: Focused on authentic culinary experiences"
        )
    )
    
    print(f"✅ Agent 1: {style_classifier.name} (Style Classifier) created successfully")
    print(f"✅ Agent 2: {culture_expert.name} (Culture Expert) created successfully")
    print(f"✅ Agent 3: {activity_expert.name} (Activity Expert) created successfully")
    print(f"✅ Agent 4: {relaxation_expert.name} (Relaxation Expert) created successfully")
    print(f"✅ Agent 5: {gourmet_expert.name} (Gourmet Expert) created successfully")
    print("="*70 + "\n")
    
    return style_classifier, culture_expert, activity_expert, relaxation_expert, gourmet_expert

# Create Conditional Agents
style_classifier, culture_expert, activity_expert, relaxation_expert, gourmet_expert = create_conditional_agents()

# ========================================================================
# Step 1: Define Executor Nodes (Conditional Branching)
# ========================================================================

@executor(id="style_classifier")
async def style_classifier_node(msg: TravelStyleRequest, ctx: WorkflowContext[TravelStyleRequest]) -> None:
    """Step 1: Classify travel style and route accordingly"""
    print(f"\n🔍 Step 1: Style Classifier - Classifying travel style.")
    print(f"   Input: {msg.user_query}")
    
    query = f"""Analyze the following travel request and classify the primary travel style into one word:

Request: {msg.user_query}

Classify it into one of the following:
- Culture: Focused on culture/history (museums, historical sites, traditions)
- Activity: Focused on activities (sports, adventure, outdoor activities)
- Relaxation: Focused on leisure (resorts, spas, relaxation)
- Gourmet: Focused on gastronomy (food tours, cooking experiences, wineries)

Respond precisely with only one word among 'Culture', 'Activity', 'Relaxation', or 'Gourmet'."""
    
    # Execute Style Classifier Agent
    thread = style_classifier.get_new_thread()
    result = await style_classifier.run(query, thread=thread)
    classification = result.text if hasattr(result, 'text') else str(result)
    
    # Process classification result
    style = classification.strip()
    if "Culture" in style or "History" in style or "Museum" in style:
        style = "Culture"
        target_id = "culture_expert"
    elif "Activity" in style or "Adventure" in style or "Sports" in style:
        style = "Activity"
        target_id = "activity_expert"
    elif "Relaxation" in style or "Resort" in style or "Healing" in style:
        style = "Relaxation"
        target_id = "relaxation_expert"
    elif "Gourmet" in style or "Food" in style or "Cuisine" in style:
        style = "Gourmet"
        target_id = "gourmet_expert"
    else:
        style = "Culture"  # Default value
        target_id = "culture_expert"
    
    # Save style in context and route to the respective expert
    msg.travel_style = style
    await ctx.send_message(msg, target_id=target_id)
    print(f"✅ Classification complete: {style} → Routing to {target_id}")


@executor(id="culture_expert")
async def culture_expert_node(msg: TravelStyleRequest, ctx: WorkflowContext[TravelStyleRequest]) -> None:
    """Step 2a: Culture/History Expert - Customized Plan"""
    print(f"\n🏛️  Step 2a: Culture Expert - Preparing customized culture/history plan.")
    
    query = f"""Create a detailed travel plan matching the culture/history travel style:

Request: {msg.user_query}

Focus points: Museums, art galleries, historical sites, and traditional cultural experiences

Include the following:
- Recommended places and activities (at least 5)
- Historical significance and highlights of each place
- Visiting times and durations
- Practical tips and cautions
- Budget guide

Craft a plan perfectly aligned with the culture/history travel style."""
    
    # Execute Culture Expert Agent
    thread = culture_expert.get_new_thread()
    result = await culture_expert.run(query, thread=thread)
    response = result.text if hasattr(result, 'text') else str(result)
    
    # Save final result and print it
    msg.specialized_plan = response
    await ctx.yield_output(msg)
    print(f"✅ Culture Expert plan complete")


@executor(id="activity_expert")
async def activity_expert_node(msg: TravelStyleRequest, ctx: WorkflowContext[TravelStyleRequest]) -> None:
    """Step 2b: Activity Expert - Customized Plan"""
    print(f"\n🏃 Step 2b: Activity Expert - Preparing customized activity plan.")
    
    query = f"""Create a detailed travel plan matching the activity travel style:

Request: {msg.user_query}

Focus points: Outdoor activities, adventure sports, hiking, and water sports

Include the following:
- Recommended activities and places (at least 5)
- Difficulty level and required equipment for each activity
- Duration and physical fitness level
- Safety rules and precautions
- Booking information and budget guide

Craft a plan perfectly aligned with the activity travel style."""
    
    # Execute Activity Expert Agent
    thread = activity_expert.get_new_thread()
    result = await activity_expert.run(query, thread=thread)
    response = result.text if hasattr(result, 'text') else str(result)
    
    # Save final result and print it
    msg.specialized_plan = response
    await ctx.yield_output(msg)
    print(f"✅ Activity Expert plan complete")


@executor(id="relaxation_expert")
async def relaxation_expert_node(msg: TravelStyleRequest, ctx: WorkflowContext[TravelStyleRequest]) -> None:
    """Step 2c: Relaxation Expert - Customized Plan"""
    print(f"\n🏖️  Step 2c: Relaxation Expert - Preparing customized relaxation plan.")
    
    query = f"""Create a detailed travel plan matching the relaxation travel style:

Request: {msg.user_query}

Focus points: Resorts, spas, wellness programs, and tranquil spots

Include the following:
- Recommended relaxation destinations and programs (at least 5)
- Features of each location and amenities available
- Wellness activities (spa, meditation, yoga, etc.)
- Accommodation details and booking methods
- Budget guide

Craft a plan perfectly aligned with the relaxation travel style."""
    
    # Execute Relaxation Expert Agent
    thread = relaxation_expert.get_new_thread()
    result = await relaxation_expert.run(query, thread=thread)
    response = result.text if hasattr(result, 'text') else str(result)
    
    # Save final result and print it
    msg.specialized_plan = response
    await ctx.yield_output(msg)
    print(f"✅ Relaxation Expert plan complete")


@executor(id="gourmet_expert")
async def gourmet_expert_node(msg: TravelStyleRequest, ctx: WorkflowContext[TravelStyleRequest]) -> None:
    """Step 2d: Gastronomy Expert - Customized Plan"""
    print(f"\n🍽️  Step 2d: Gourmet Expert - Preparing customized gastronomy plan.")
    
    query = f"""Create a detailed travel plan matching the gastronomy travel style:

Request: {msg.user_query}

Focus points: Michelin restaurants, food tours, cooking experiences, and wine tasting

Include the following:
- Recommended restaurants and gastronomic activities (at least 5)
- Signature menus and unique features of each restaurant
- Itineraries for culinary experiences and food market explorations
- Reservation methods, dress codes, and price ranges
- Gourmet tour budget guide

Craft a plan perfectly aligned with the gastronomy travel style."""
    
    # Execute Gastronomy Expert Agent
    thread = gourmet_expert.get_new_thread()
    result = await gourmet_expert.run(query, thread=thread)
    response = result.text if hasattr(result, 'text') else str(result)
    
    # Save final result and print it
    msg.specialized_plan = response
    await ctx.yield_output(msg)
    print(f"✅ Gourmet Expert plan complete")


# ========================================================================
# Step 2: Build Conditional Branching Graph with WorkflowBuilder
# ========================================================================

conditional_workflow = (
    WorkflowBuilder()
    # Start at Style Classifier
    .set_start_executor(style_classifier_node)
    # Classifier → Conditionally route to 4 expert nodes (only one will execute)
    .add_edge(style_classifier_node, culture_expert_node)
    .add_edge(style_classifier_node, activity_expert_node)
    .add_edge(style_classifier_node, relaxation_expert_node)
    .add_edge(style_classifier_node, gourmet_expert_node)
    .build()
)

print("\n" + "="*70)
print("✅ Conditional Workflow Graph successfully built")
print("="*70)
print("📊 Workflow structure (Conditional Branching Pattern):")
print("   style_classifier_node (Classification)")
print("         ↓")
print("   [Conditional Routing]")
print("   ↙  ↓  ↓  ↘")
print(" Culture Activity Relaxation Gourmet")
print(" Expert    Expert     Expert     Expert")
print("="*70 + "\n")


# ========================================================================
# Step 3: Execute Workflow
# ========================================================================

async def run_conditional_workflow():
    """Function to execute Conditional Workflow"""
    print("\n" + "="*70)
    print("🚀 Starting Conditional Workflow Execution (MAF WorkflowBuilder)")
    print("="*70)

    # Create Input Message
    user_query = "Planning a 5-day trip to London with a focus on exploring historical architecture and museums. The British Museum and Tower Bridge are a must-see."
    style_request = TravelStyleRequest(user_query=user_query)

    # Execute Workflow (run_stream - async generator)
    outputs = []
    async for event in conditional_workflow.run_stream(style_request):
        # Extract output from event
        if hasattr(event, 'output') and event.output is not None:
            outputs.append(event.output)
            print(f"📤 Received event: {type(event.output).__name__}")
        elif hasattr(event, 'data') and event.data is not None:
            outputs.append(event.data)
            print(f"📤 Received event: {type(event.data).__name__}")

    # Extract final result
    if outputs:
        final_result = outputs[-1]
    else:
        raise ValueError("❌ No output received from Workflow.")

    print("\n" + "="*70)
    print("📊 Conditional Pattern Results (MAF WorkflowBuilder)")
    print("="*70)
    print(f"\n📝 Travel Request: {final_result.user_query}\n")
    print(f"🎯 Classified Travel Style: {final_result.travel_style}\n")
    print(f"\n{'='*70}")
    print(f"📋 Customized Travel Plan:")
    print(f"{'='*70}")
    print(f"{final_result.specialized_plan}")
    print(f"\n{'='*70}")
    
    return final_result

# Execute Workflow
conditional_result = await run_conditional_workflow()

**Conditional Branching Pattern Analysis (Using MAF WorkflowBuilder)**

### ✅ Key Components

**1. Message Type (Data Transmission)**
- Stores travel style and plan in the `TravelStyleRequest` dataclass
- `travel_style`: Classified styles ('Culture', 'Activity', 'Relaxation', 'Gourmet')
- `specialized_plan`: Customized plan by selected expert

**2. Executor Node (@executor decorator)**
- `@executor(id="style_classifier")`: Step 1 - Classifies travel style and assigns routing
- `@executor(id="culture_expert")`: Step 2a - Culture/History expert plan
- `@executor(id="activity_expert")`: Step 2b - Activity expert plan
- `@executor(id="relaxation_expert")`: Step 2c - Relaxation expert plan
- `@executor(id="gourmet_expert")`: Step 2d - Gourmet expert plan

**3. WorkflowBuilder (Constructing Conditional Branching Graphs)**
```python
workflow = (
    WorkflowBuilder()
    .set_start_executor(style_classifier_node)  # Start with classifier
    # Classifier → Connect to 4 expert nodes (conditional routing)
    .add_edge(style_classifier_node, culture_expert_node)
    .add_edge(style_classifier_node, activity_expert_node)
    .add_edge(style_classifier_node, relaxation_expert_node)
    .add_edge(style_classifier_node, gourmet_expert_node)
    .build()
)
```

**4. Dynamic Routing Mechanism**
- `ctx.send_message(msg, target_id=selected_expert)`: Sends only to specific nodes based on conditions
- One of the 4 expert nodes is executed based on classification results

### ✅ Execution Flow

```
TravelStyleRequest creation
    ↓
style_classifier_node
(Foundry Agent: style_classifier)
    → Analyze and classify travel style
    ↓
[Conditional Routing]
    ↙  ↓  ↓  ↘
Culture   Activity   Relaxation   Gourmet
Expert    Expert     Expert       Expert
(Only one is executed)
    ↓
specialized_plan creation
    ↓
Final result output
```

### ✅ Advantages of MAF WorkflowBuilder Conditional Pattern

1. **Dynamic Routing**: Selects next node at runtime using `send_message(target_id=...)`
2. **Scalability**: Easy to add new styles/experts
3. **Type Safety**: Explicit message structure defined with dataclass
4. **Efficiency**: Executes only required expert (out of 4)
5. **Maintainability**: Each expert node is managed independently

### ✅ Implementation Methods for Conditional Branching

| Method                     | Description                   | Pros                       | Cons                        |
|----------------------------|-------------------------------|----------------------------|-----------------------------|
| **send_message(target_id)** | Sends messages to specific nodes only | Clear control, efficient  | Requires manual routing logic |
| **add_edge + Conditional Execution** | Defines all edges, checks conditions in nodes | Explicit graph structure   | Nodes are highly interconnected |

### ✅ Use Cases
- Service Routing Based on User Intent
- Tier Classification by Complexity/Priority
- Assign Experts for Specialized Domains
- Personalized Recommendation Systems

**Architecture Summary**:
```
User Request
    ↓
Classification Agent (Determine Style)
    ↓
  Conditional Branching
  ↙ ↓ ↓ ↘
Culture Activity Relaxation Gourmet
Expert  Expert     Expert     Expert
  ↘ ↓ ↓ ↙
 Custom Plan Creation
```

**Real-World Example**:
- Input: "I want to deeply experience Europe's history and art."
- Classification: Automatically classified as "Culture" style
- Routing: Delivered to the Culture Expert (other 3 experts are not executed)
- Output: A detailed plan centered on museums, galleries, and historical sites

## 5. Loop-Based Pattern (Loop-Based Workflow)

In [ ]:
"""
Loop-Based Pattern - Using MAF WorkflowBuilder
- Define iterative loop nodes with the @executor decorator
- Construct loop graph with WorkflowBuilder
- Conditional iteration (up to 3 times) for itinerary optimization
"""

# ========================================================================
# Define Message Type (for Loop Pattern)
# ========================================================================

@dataclass
class ItineraryRefinementRequest:
    """Message requesting iterative refinement of travel itinerary"""
    destination: str
    days: int
    current_itinerary: Optional[str] = None
    feedback: Optional[str] = None
    feedback_history: Optional[List[str]] = None
    iteration_count: int = 0
    max_iterations: int = 3
    final_itinerary: Optional[str] = None

# ========================================================================
# Create Loop Pattern Agents
# ========================================================================

def create_loop_agents():
    """Create agents for iterative improvement"""
    
    if not agent_client:
        raise ValueError("❌ MAF Agent client has not been initialized.")
    
    print("\n" + "="*70)
    print("🤖 Creating loop pattern agents.")
    print("="*70)
    
    # Agent 1: Itinerary Generator (Initial creation/improvement)
    itinerary_generator = agent_client.create_agent(
        name="ItineraryGenerator",
        instructions=(
            "You are an expert in creating travel itineraries.\n\n"
            "Role:\n"
            "- Create an initial travel itinerary or refine it based on feedback\n"
            "- Draft detailed daily schedules\n"
            "- Include major tourist attractions and activities\n"
            "- Recommend meals and provide travel time guidance\n"
            "- Suggest estimated costs\n"
            "- Always incorporate feedback for improvements if provided\n\n"
            "Style: Detailed and actionable itineraries"
        )
    )
    
    # Agent 2: Feedback Analyzer (Analyze feedback)
    feedback_analyzer = agent_client.create_agent(
        name="FeedbackAnalyzer",
        instructions=(
            "You are an expert in reviewing travel itineraries.\n\n"
            "Role:\n"
            "- Evaluate the proposed travel itinerary from multiple perspectives\n"
            "- Assess the appropriateness of time allocation\n"
            "- Analyze efficiency of travel routes\n"
            "- Review diversity of activities\n"
            "- Ensure sufficient rest time\n"
            "- Assess budget feasibility\n"
            "- Provide specific and actionable improvement suggestions\n\n"
            "Style: Constructive and specific feedback"
        )
    )
    
    # Agent 3: Final Optimizer (Final optimization)
    final_optimizer = agent_client.create_agent(
        name="FinalOptimizer",
        instructions=(
            "You are an expert in the final review of travel itineraries.\n\n"
            "Role:\n"
            "- Synthesize all feedback to create the optimized final itinerary\n"
            "- Summarize the improvement process\n"
            "- Provide the final detailed daily schedule\n"
            "- Highlight key attractions and features\n"
            "- Offer practical tips\n"
            "- Summarize the estimated total costs\n\n"
            "Style: Complete and detailed final plans"
        )
    )
    
    print(f"✅ Agent 1: {itinerary_generator.name} (Itinerary Generator) successfully created")
    print(f"✅ Agent 2: {feedback_analyzer.name} (Feedback Analyzer) successfully created")
    print(f"✅ Agent 3: {final_optimizer.name} (Final Optimizer) successfully created")
    print("="*70 + "\n")
    
    return itinerary_generator, feedback_analyzer, final_optimizer

# Create Loop Agents
itinerary_generator, feedback_analyzer, final_optimizer = create_loop_agents()

# ========================================================================
# Step 1: Define Executor Nodes (Loop Pattern)
# ========================================================================

@executor(id="itinerary_generator")
async def itinerary_generator_node(msg: ItineraryRefinementRequest, ctx: WorkflowContext[ItineraryRefinementRequest]) -> None:
    """Step 1: Create/Refine travel itinerary"""
    msg.iteration_count += 1
    is_initial = msg.iteration_count == 1
    
    print(f"\n📋 Step 1: Itinerary Generator - {'Initial itinerary creation' if is_initial else f'Itinerary refinement (Round {msg.iteration_count})'}.")
    
    if is_initial:
        query = f"""Please generate the initial travel itinerary.

Destination: {msg.destination}
Duration: {msg.days} days

The itinerary must include:
- Detailed daily schedule
- Major tourist attractions and activities
- Meal recommendations
- Travel time and mode
- Estimated costs

Please provide a detailed and actionable itinerary."""
    else:
        query = f"""Please propose an updated itinerary based on the feedback provided.

Destination: {msg.destination}
Duration: {msg.days} days

Previous itinerary:
{msg.current_itinerary[:500]}.

Recent feedback:
{msg.feedback[:500]}.

Ensure the feedback is incorporated to improve the itinerary."""
    
    # Execute Itinerary Generator Agent
    thread = itinerary_generator.get_new_thread()
    result = await itinerary_generator.run(query, thread=thread)
    response = result.text if hasattr(result, 'text') else str(result)
    
    # Update itinerary
    msg.current_itinerary = response
    
    print(f"✅ {'Initial itinerary' if is_initial else f'Refined itinerary (Round {msg.iteration_count})'} successfully created")
    print(f"\nPreview of generated itinerary:\n{response[:300]}.\n")
    
    # Check maximum iteration count
    if msg.iteration_count >= msg.max_iterations:
        # Move to final optimization
        await ctx.send_message(msg, target_id="final_optimizer")
    else:
        # Move to feedback analysis
        await ctx.send_message(msg, target_id="feedback_analyzer")


@executor(id="feedback_analyzer")
async def feedback_analyzer_node(msg: ItineraryRefinementRequest, ctx: WorkflowContext[ItineraryRefinementRequest]) -> None:
    """Step 2: Feedback analysis and improvement suggestions"""
    print(f"\n💭 Step 2: Feedback Analyzer - Reviewing itinerary (Round {msg.iteration_count}).")
    
    query = f"""Please review the following itinerary and propose improvements:

Destination: {msg.destination}, {msg.days} days

Current itinerary:
{msg.current_itinerary[:600]}.

Review based on the following perspectives:
1. Appropriateness of time allocation
2. Efficiency of travel routes
3. Diversity of activities
4. Adequate rest time
5. Budget feasibility

"""
    
    # Execute Feedback Analyzer Agent
    thread = feedback_analyzer.get_new_thread()
    result = await feedback_analyzer.run(query, thread=thread)
    response = result.text if hasattr(result, 'text') else str(result)
    
    # Save feedback
    msg.feedback = response
    if msg.feedback_history is None:
        msg.feedback_history = []
    msg.feedback_history.append(response)
    
    print(f"✅ Feedback analysis completed (Round {msg.iteration_count})")
    print(f"\nPreview of feedback:\n{response[:300]}.\n")
    
    # Loop back to itinerary creation for refinement
    await ctx.send_message(msg, target_id="itinerary_generator")


@executor(id="final_optimizer")
async def final_optimizer_node(msg: ItineraryRefinementRequest, ctx: WorkflowContext[ItineraryRefinementRequest]) -> None:
    """Step 3: Final optimization and completion"""
    print(f"\n🎯 Step 3: Final Optimizer - Performing final optimization.")
    
    feedback_summary = "\n".join([f"- Round {i+1}: {fb[:200]}." 
                                   for i, fb in enumerate(msg.feedback_history or [])])
    
    query = f"""Based on all feedback received, please provide the optimized final itinerary.

Destination: {msg.destination}
Duration: {msg.days} days
Improvement iterations: {msg.iteration_count} rounds

Last itinerary:
{msg.current_itinerary[:600]}.

Improvement feedback history:
{feedback_summary}

The final itinerary must include:
✅ Summary of the improvement process
✅ Final detailed daily schedule
✅ Key highlights
✅ Practical tips
✅ Estimated total costs

"""
    
    # Execute Final Optimizer Agent
    thread = final_optimizer.get_new_thread()
    result = await final_optimizer.run(query, thread=thread)
    response = result.text if hasattr(result, 'text') else str(result)
    
    # Save final result and output
    msg.final_itinerary = response
    await ctx.yield_output(msg)
    print(f"✅ Final optimization successfully completed")


# ========================================================================
# Step 2: Construct Loop Graph with WorkflowBuilder
# ========================================================================

loop_workflow = (
    WorkflowBuilder()
    # Start from Itinerary Generator
    .set_start_executor(itinerary_generator_node)
    # Generator → Feedback Analyzer (conditional: iteration < max)
    .add_edge(itinerary_generator_node, feedback_analyzer_node)
    # Feedback Analyzer → Generator (loop back)
    .add_edge(feedback_analyzer_node, itinerary_generator_node)
    # Generator → Final Optimizer (conditional: iteration >= max)
    .add_edge(itinerary_generator_node, final_optimizer_node)
    .build()
)

print("\n" + "="*70)
print("✅ Loop-Based Workflow graph successfully constructed")
print("="*70)
print("📊 Workflow structure (Loop Pattern):")
print("   itinerary_generator_node (Start)")
print("         ↓")
print("   [iteration < max?]")
print("         ↓ Yes")
print("   feedback_analyzer_node")
print("         ↓")
print("   [Loop back]")
print("         ↓")
print("   itinerary_generator_node")
print("         ↓ (Repeat)")
print("   [iteration >= max?]")
print("         ↓ Yes")
print("   final_optimizer_node (Complete)")
print("="*70 + "\n")


# ========================================================================
# Step 3: Execute Workflow
# ========================================================================

async def run_loop_workflow():
    """Function to execute Loop Workflow"""
    print("\n" + "="*70)
    print("🚀 Starting Loop-Based Workflow execution (MAF WorkflowBuilder)")
    print("="*70)
    print(f"🔄 Maximum iterations: 3 rounds")
    print(f"📌 Pattern: Create itinerary → Feedback → Improvement → (Repeat) → Final Optimization")
    print("="*70)

    # Create input message
    refinement_request = ItineraryRefinementRequest(
        destination="London",
        days=5,
        max_iterations=3
    )

    # Execute Workflow (with run_stream - async generator)
    outputs = []
    async for event in loop_workflow.run_stream(refinement_request):
        # Extract output from event
        if hasattr(event, 'output') and event.output is not None:
            outputs.append(event.output)
            print(f"📤 Event received: {type(event.output).__name__}")
        elif hasattr(event, 'data') and event.data is not None:
            outputs.append(event.data)
            print(f"📤 Event received: {type(event.data).__name__}")

    # Extract final result
    if outputs:
        final_result = outputs[-1]
    else:
        raise ValueError("❌ No output received from the workflow.")

    print("\n" + "="*70)
    print("📊 Loop-Based Pattern Results (MAF WorkflowBuilder)")
    print("="*70)
    print(f"\n📝 Destination: {final_result.destination}")
    print(f"📅 Duration: {final_result.days} days")
    print(f"🔄 Improvement iterations: {final_result.iteration_count} rounds")
    print(f"💭 Feedback history: {len(final_result.feedback_history or [])} items")
    
    print(f"\n{'='*70}")
    print(f"📋 Final optimized itinerary:")
    print(f"{'='*70}")
    print(f"{final_result.final_itinerary}")
    print(f"\n{'='*70}")
    
    if final_result.feedback_history:
        print(f"\n{'='*70}")
        print(f"💭 Improvement feedback history:")
        print(f"{'='*70}")
        for i, feedback in enumerate(final_result.feedback_history, 1):
            print(f"\n[Round {i} feedback]")
            print(feedback[:400] + "." if len(feedback) > 400 else feedback)
        print(f"\n{'='*70}")
    
    return final_result

# Execute the Workflow
loop_result = await run_loop_workflow()

**Loop-Based Pattern Analysis (Using MAF WorkflowBuilder)**

### ✅ Key Components

**1. Message Type (Data Transfer and State Management)**
- Use the `ItineraryRefinementRequest` dataclass to track loop states.
- `current_itinerary`: current itinerary (updated on each iteration)
- `feedback`: latest feedback
- `feedback_history`: history of all feedback
- `iteration_count`: current loop iteration count
- `max_iterations`: maximum loop iteration count (set to 3)

**2. Executor Nodes (@executor decorator)**
- `@executor(id="itinerary_generator")`: Generates or refines itineraries (supports looping).
- `@executor(id="feedback_analyzer")`: Analyzes feedback (supports looping).
- `@executor(id="final_optimizer")`: Performs final optimization (ends the loop).

**3. WorkflowBuilder (Constructing the Loop Graph)**
```python
workflow = (
    WorkflowBuilder()
    .set_start_executor(itinerary_generator_node)
    # Loop structure: Generator ↔ Analyzer
    .add_edge(itinerary_generator_node, feedback_analyzer_node)
    .add_edge(feedback_analyzer_node, itinerary_generator_node)
    # Exit condition: Generator → Optimizer
    .add_edge(itinerary_generator_node, final_optimizer_node)
    .build()
)
```

**4. Loop Control Mechanism**
- Tracking via `iteration_count` increment.
- Conditional routing:
  - `iteration < max` → Feedback Analyzer (continue looping)
  - `iteration >= max` → Final Optimizer (end loop)

### ✅ Execution Flow

```
ItineraryRefinementRequest initialization (iteration=0)
    ↓
itinerary_generator_node (iteration=1)
    → Generate initial itinerary
    ↓
[iteration < max?] Yes
    ↓
feedback_analyzer_node
    → Generate feedback
    ↓
[Loop-back] itinerary_generator_node (iteration=2)
    → Improve itinerary based on feedback
    ↓
[iteration < max?] Yes
    ↓
feedback_analyzer_node
    → Generate feedback
    ↓
[Loop-back] itinerary_generator_node (iteration=3)
    → Improve itinerary based on feedback
    ↓
[iteration >= max?] Yes
    ↓
final_optimizer_node
    → Optimize final itinerary
    ↓
Output final result
```

### ✅ Advantages of MAF WorkflowBuilder Loop Pattern

1. **Explicit loop structure**: Loop paths defined through edges.
2. **State tracking**: Progress managed using the dataclass `iteration_count`.
3. **Conditional termination**: Node-level condition checks enable routing to distinct paths.
4. **History tracking**: Improvement processes recorded in `feedback_history`.
5. **Flexibility**: Adjust loop iteration count by modifying `max_iterations`.

### ✅ Loop Implementation Pattern

| Component    | Role                | Implementation              |
|--------------|---------------------|-----------------------------|
| **Counter**  | Track iteration count | `msg.iteration_count += 1` |
| **Exit Condition** | Decide loop termination | `if iteration >= max` |
| **Loop-back** | Return to starting node | `send_message(target_id="generator")` |
| **State Transfer** | Preserve previous output | Update fields in dataclass |

### ✅ Use Cases
- Document drafting with iterative improvements.
- Code reviews and revision cycles.
- Iterative design feedback application.
- Quality validation and enhancement.
- Learning and optimization processes.

**Architecture Summary**:
```
Initial Request
    ↓
┌──→ Itinerary Generation ─→ [Exit?] ─Yes→ Final Optimization
│            ↓                   ↑
│           No                   │
│            ↓                   │
└── Feedback Analysis ───────────┘
       (Loop-back)
```

**Example Flow**:
- Input: "5-day London travel itinerary"
- Round 1: Generate initial itinerary → Feedback (needs better route optimization)
- Round 2: Improved itinerary with optimized route → Feedback (add rest periods)
- Round 3: Itinerary with added rest → Final optimization
- Output: Final optimized itinerary after 3 iterations of improvement.

---

## 6. Error Handling & Retry Pattern

In [ ]:
"""
Error Handling Pattern - Using MAF WorkflowBuilder
- Define error handling nodes using the @executor decorator
- Build error recovery graph with WorkflowBuilder
- Error detection → Analysis → Suggest alternatives → Recovery
"""

# ========================================================================
# Define Message Type (for Error Handling Pattern)
# ========================================================================

@dataclass
class BookingRequest:
    """Booking request message (including error handling)"""
    destination: str
    dates: str
    hotel_name: str
    simulate_error: bool = False
    booking_result: Optional[str] = None
    error_detected: bool = False
    error_analysis: Optional[str] = None
    fallback_options: Optional[str] = None
    final_booking: Optional[str] = None
    status: str = "pending"

# ========================================================================
# Create Error Handling Pattern Agents
# ========================================================================

def create_error_handling_agents():
    """Create agents for error handling"""
    
    if not agent_client:
        raise ValueError("❌ MAF Agent client not initialized.")
    
    print("\n" + "="*70)
    print("🤖 Creating agents for error handling pattern.")
    print("="*70)
    
    # Agent 1: Booking Agent (Attempt booking)
    booking_agent = agent_client.create_agent(
        name="BookingAgent",
        instructions=(
            "You are a travel booking expert.\n\n"
            "Role:\n"
            "- Process hotel and flight bookings\n"
            "- Check availability\n"
            "- Provide hotel information and pricing\n"
            "- Provide detailed booking results\n"
            "- Communicate any specifics or important details\n\n"
            "Style: Accurate and detailed booking information"
        )
    )
    
    # Agent 2: Error Analyzer
    error_analyzer = agent_client.create_agent(
        name="ErrorAnalyzer",
        instructions=(
            "You are an expert in analyzing booking system errors.\n\n"
            "Role:\n"
            "- Conduct in-depth analysis of booking failure reasons\n"
            "- Categorize type of failures (e.g., sold out, price fluctuations, system errors)\n"
            "- Assess feasibility of alternatives\n"
            "- Recommend solutions\n"
            "- Assess customer impact\n\n"
            "Style: Systematic and constructive analysis"
        )
    )
    
    # Agent 3: Fallback Planner (Suggest Alternatives)
    fallback_planner = agent_client.create_agent(
        name="FallbackPlanner",
        instructions=(
            "You are an expert in planning alternative travel options.\n\n"
            "Role:\n"
            "- Propose practical alternatives for failed bookings\n"
            "- Recommend other hotels in the same area (at least three suggestions)\n"
            "- Suggest date change options\n"
            "- Offer nearby alternative city options\n"
            "- Compare pros and cons of each alternative\n"
            "- Include price and quality details\n\n"
            "Style: Specific and actionable alternatives"
        )
    )
    
    # Agent 4: Recovery Agent (Recovery and Rebooking)
    recovery_agent = agent_client.create_agent(
        name="RecoveryAgent",
        instructions=(
            "You are an expert in booking recovery.\n\n"
            "Role:\n"
            "- Select and recommend the best alternative\n"
            "- Proceed with rebooking and finalization\n"
            "- Summarize the final booking details\n"
            "- Prepare customer guidance\n"
            "- Explain differences compared to the original plan\n\n"
            "Style: Reassuring and clear final guidance"
        )
    )
    
    print(f"✅ Agent 1: {booking_agent.name} (Booking Agent) created successfully")
    print(f"✅ Agent 2: {error_analyzer.name} (Error Analyzer) created successfully")
    print(f"✅ Agent 3: {fallback_planner.name} (Fallback Planner) created successfully")
    print(f"✅ Agent 4: {recovery_agent.name} (Recovery Agent) created successfully")
    print("="*70 + "\n")
    
    return booking_agent, error_analyzer, fallback_planner, recovery_agent

# Generate Error Handling Agents
booking_agent, error_analyzer, fallback_planner, recovery_agent = create_error_handling_agents()

# ========================================================================
# Step 1: Define Executor Nodes (for Error Handling Pattern)
# ========================================================================

@executor(id="booking_agent")
async def booking_agent_node(msg: BookingRequest, ctx: WorkflowContext[BookingRequest]) -> None:
    """Step 1: Attempt booking"""
    print(f"\n🔄 Step 1: Booking Agent - Attempting booking.")
    
    if msg.simulate_error:
        query = f"""[Error Simulation] The following booking has failed:

Destination: {msg.destination}
Dates: {msg.dates}
Hotel: {msg.hotel_name}

Please explain the reason why the booking is not possible:
- Fully booked
- Price increase
- Other reasons

"""
    else:
        query = f"""Please process the following booking:

Destination: {msg.destination}
Dates: {msg.dates}
Hotel: {msg.hotel_name}

Provide the booking results in this format:
- Availability: Available/Unavailable
- Hotel Info: Rating, location, features
- Price Info: Price per night, total cost
- Notes: Breakfast included, cancellation policy, etc.

"""
    
    # Run Booking Agent
    thread = booking_agent.get_new_thread()
    result = await booking_agent.run(query, thread=thread)
    response = result.text if hasattr(result, 'text') else str(result)
    
    # Save booking results
    msg.booking_result = response
    
    print(f"📋 Preview of booking results:\n{response[:300]}.\n")
    
    # Error detection (based on keywords)
    error_keywords = ["unavailable", "failed", "fully booked", "not available", "cannot"]
    msg.error_detected = any(keyword in response for keyword in error_keywords)
    
    if msg.error_detected:
        print(f"⚠️  Error detected! Forwarding to Error Analyzer")
        msg.status = "error_detected"
        await ctx.send_message(msg, target_id="error_analyzer")
    else:
        print(f"✅ Booking successful!")
        msg.status = "success"
        await ctx.yield_output(msg)


@executor(id="error_analyzer")
async def error_analyzer_node(msg: BookingRequest, ctx: WorkflowContext[BookingRequest]) -> None:
    """Step 2: Analyze error"""
    print(f"\n🔍 Step 2: Error Analyzer - Analyzing reasons for failure.")
    
    query = f"""Please conduct a thorough analysis of the reasons for the booking failure and suggest directions for alternatives.

Failed Booking Info:
- Destination: {msg.destination}
- Dates: {msg.dates}
- Hotel: {msg.hotel_name}

Failure Situation:
{msg.booking_result[:500]}.

Analyze the following:
1. Main failure cause (e.g., fully booked, price hike, system error)
2. Categorization of failure type
3. Feasibility of alternatives
4"""
    
    # Run Error Analyzer Agent
    thread = error_analyzer.get_new_thread()
    result = await error_analyzer.run(query, thread=thread)
    response = result.text if hasattr(result, 'text') else str(result)
    
    # Save analysis results
    msg.error_analysis = response
    msg.status = "error_analyzed"
    
    print(f"✅ Error analysis completed")
    print(f"Preview of analysis results:\n{response[:300]}.\n")
    
    # Forward to Fallback Planner
    await ctx.send_message(msg, target_id="fallback_planner")


@executor(id="fallback_planner")
async def fallback_planner_node(msg: BookingRequest, ctx: WorkflowContext[BookingRequest]) -> None:
    """Step 3: Suggest alternative options"""
    print(f"\n🎯 Step 3: Fallback Planner - Suggesting alternative options.")
    
    query = f"""Please provide at least three specific alternatives for the failed booking.

Original Plan:
- Destination: {msg.destination}
- Dates: {msg.dates}
- Hotel: {msg.hotel_name}

Analysis of Failure Reasons:
{msg.error_analysis[:400]}.

Propose the following alternatives:
1. Other hotels in the same area (at least 3 options with various price ranges)
2. Date change options (if possible)
3. Alternative nearby city options
4"""
    
    # Run Fallback Planner Agent
    thread = fallback_planner.get_new_thread()
    result = await fallback_planner.run(query, thread=thread)
    response = result.text if hasattr(result, 'text') else str(result)
    
    # Save alternatives
    msg.fallback_options = response
    msg.status = "fallback_planned"
    
    print(f"✅ Alternatives suggested successfully")
    print(f"Preview of alternatives:\n{response[:300]}.\n")
    
    # Forward to Recovery Agent
    await ctx.send_message(msg, target_id="recovery_agent")


@executor(id="recovery_agent")
async def recovery_agent_node(msg: BookingRequest, ctx: WorkflowContext[BookingRequest]) -> None:
    """Step 4: Select the best alternative and recover"""
    print(f"\n✅ Step 4: Recovery Agent - Selecting the best alternative and rebooking.")
    
    query = f"""Select the best alternative and complete the rebooking process.

Original Plan:
- Destination: {msg.destination}
- Dates: {msg.dates}
- Hotel: {msg.hotel_name}

Proposed Alternatives:
{msg.fallback_options[:600]}.

Perform the following:
1. Select the best alternative and explain the selection rationale
2. Rebooking progress (simulate completion)
3. Final booking details (hotel name, price, features)
4. Explain differences from the original plan
5"""
    
    # Run Recovery Agent
    thread = recovery_agent.get_new_thread()
    result = await recovery_agent.run(query, thread=thread)
    response = result.text if hasattr(result, 'text') else str(result)
    
    # Save final recovery result and output
    msg.final_booking = response
    msg.status = "recovered"
    
    print(f"✅ Recovery complete")
    
    await ctx.yield_output(msg)


# ========================================================================
# Step 2: Build Error Handling Graph with WorkflowBuilder
# ========================================================================

error_handling_workflow = (
    WorkflowBuilder()
    # Start with Booking Agent
    .set_start_executor(booking_agent_node)
    # Booking Agent → Error Analyzer (if error occurs)
    .add_edge(booking_agent_node, error_analyzer_node)
    # Error Analyzer → Fallback Planner
    .add_edge(error_analyzer_node, fallback_planner_node)
    # Fallback Planner → Recovery Agent
    .add_edge(fallback_planner_node, recovery_agent_node)
    .build()
)

print("\n" + "="*70)
print("✅ Error Handling Workflow graph built successfully")
print("="*70)
print("📊 Workflow structure (Error Handling Pattern):")
print("   booking_agent_node (Attempt booking)")
print("         ↓")
print("   [Error detected?]")
print("         ↓ Yes")
print("   error_analyzer_node (Analyze error)")
print("         ↓")
print("   fallback_planner_node (Propose alternatives)")
print("         ↓")
print("   recovery_agent_node (Recovery)")
print("="*70 + "\n")


# ========================================================================
# Step 3: Run Workflow
# ========================================================================

async def run_error_handling_workflow(simulate_error: bool = True):
    """Function to run Error Handling Workflow"""
    print("\n" + "="*70)
    print("🚀 Starting Error Handling Workflow (MAF WorkflowBuilder)")
    print("="*70)
    print(f"🧪 Error Simulation: {'Enabled' if simulate_error else 'Disabled'}")
    print("="*70)

    # Create input message
    booking_request = BookingRequest(
        destination="London",
        dates="2025-03-15 ~ 2025-03-19",
        hotel_name="Hilton London Paddington",
        simulate_error=simulate_error
    )

    # Execute Workflow (use run_stream - async generator)
    outputs = []
    async for event in error_handling_workflow.run_stream(booking_request):
        # Extract output from event
        if hasattr(event, 'output') and event.output is not None:
            outputs.append(event.output)
            print(f"📤 Event received: {type(event.output).__name__}")
        elif hasattr(event, 'data') and event.data is not None:
            outputs.append(event.data)
            print(f"📤 Event received: {type(event.data).__name__}")

    # Extract final result
    if outputs:
        final_result = outputs[-1]
    else:
        raise ValueError("❌ No output received from Workflow.")

    print("\n" + "="*70)
    print("📊 Error Handling Pattern Results (MAF WorkflowBuilder)")
    print("="*70)
    print(f"\n📝 Booking information:")
    print(f"   - Destination: {final_result.destination}")
    print(f"   - Dates: {final_result.dates}")
    print(f"   - Hotel: {final_result.hotel_name}")
    print(f"\n✅ Final Status: {final_result.status}")
    print(f"⚠️  Error Detected: {'Yes' if final_result.error_detected else 'No'}")
    
    if final_result.error_detected:
        print(f"\n{'='*70}")
        print(f"🔍 Error Analysis:")
        print(f"{'='*70}")
        print(f"{final_result.error_analysis}\n")
        
        print(f"\n{'='*70}")
        print(f"🎯 Alternative Options:")
        print(f"{'='*70}")
        print(f"{final_result.fallback_options}\n")
        
        print(f"\n{'='*70}")
        print(f"✅ Final Recovery Result:")
        print(f"{'='*70}")
        print(f"{final_result.final_booking}")
    else:
        print(f"\n{'='*70}")
        print(f"✅ Booking Result:")
        print(f"{'='*70}")
        print(f"{final_result.booking_result}")
    
    print(f"\n{'='*70}")
    
    return final_result

# Run workflow (error simulation enabled)
print("\n🧪 Scenario: Booking failure situation (Error handling test)")
error_result = await run_error_handling_workflow(simulate_error=True)


### Analysis and Summary of Error Handling Patterns

#### 📊 Advantages of Error Handling Patterns

1. **Automatic Failure Detection and Recovery**
   - Automatic identification of failures using keywords.
   - The agent analyzes causes and suggests alternatives.
   - Attempts automatic recovery without user intervention.
   - Prevents complete booking failures.

2. **Step-by-step Error Handling**
   - **Booking Node**: Initial attempt and failure detection.
   - **Error Analyzer**: Detailed analysis of failure causes.
   - **Fallback Planner**: Creation of alternative options.
   - **Recovery Node**: Final recovery and booking completion.

3. **Improved User Experience**
   - Transparent error messages.
   - Clear suggestions for alternatives.
   - Time-saving through automatic recovery.
   - Enhanced customer satisfaction.

4. **Enhanced System Reliability**
   - Stable operation even in exceptional scenarios.
   - Assurance of business continuity.
   - Error pattern tracking and improvement.
   - Suitable for production environments.

#### 🎯 Real-world Application Cases

**Travel Booking System**
```
Normal Flow: Attempt to book → Success → Confirmation
Failure Flow: Attempt to book → Failure → Analysis → Alternative → Recovery
```

**E-commerce**
```
Normal: Order → Payment → Delivery
Failure: Order → Out of stock → Recommend alternative product → Completion
```

**API Integration System**
```
Normal: API Call → Success
Failure: API Call → Timeout → Retry → Cache data → Success
```

**Financial Transactions**
```
Normal: Transaction Request → Approval → Completion
Failure: Transaction Request → Exceeds limit → Suggest split transaction → Approval → Completion
```

#### 💡 Tips for Implementing Error Handling Patterns

**1. Clear Definition of Failure Criteria**
```python
# Detection using keywords
error_keywords = ["failure", "impossible", "cannot book", "full", "sold out"]

# Detection using status codes
if status_code in [400, 404, 500]:
    trigger_error_handling()
```

**2. Detailed Failure Analysis**
```python
# Categorization of failure causes
- Issues with inventory/availability
- System errors
- Problems with input data
- External dependency failures
```

**3. Multi-level Fallback Strategy**
```python
Level 1: Automatic retry (3 attempts)
Level 2: Suggest alternative options
Level 3: Request user confirmation
Level 4: Escalate to manual processing
```

**4. Logging and Monitoring**
```python
logger.warning(f"Failure detected: {error_type}")
logger.info(f"Alternative options generated: {alternatives}")
logger.info(f"Recovery successful: {recovery_result}")

# Metrics collection
metrics.increment('booking.failures')
metrics.increment('booking.recoveries')
```

#### 🚨 Precautions

⚠️ **Avoid Infinite Retry Loops**
- Set a maximum number of retries (e.g., 3).
- Use exponential backoff.
- If recovery ultimately fails, provide clear error messaging.

⚠️ **Prevent Circular References**
- Ensure one-way flow: Error Analyzer → Fallback → Recovery.
- Prevent infinite loops in case of recovery failures.

⚠️ **Cost Management**
- API call costs incurred during retries.
- Set timeout limits to minimize costs.
- Keep fallback options simple.

#### 📈 Performance Metrics

| Metric            | Target  | Description                            |
|-------------------|---------|----------------------------------------|
| **Recovery Rate** | > 80%   | Automatic recovery success rate        |
| **Time to Recovery** | < 30 sec | Average recovery duration              |
| **False Positive** | < 5%   | Incorrect detection of normal as failed |
| **User Satisfaction** | > 4.5/5 | Customer satisfaction post-recovery     |

---

## 7. Handoff Pattern - Dynamic Agent Transition

### What is the Handoff Pattern?

The Handoff Pattern refers to a dynamic shift of control to **different specialized agents** based on the complexity or nature of the task.

```
Initial Agent → [Evaluation] → Condition 1? → Specialist A
                        → Condition 2? → Specialist B  
                        → Condition 3? → Specialist C
                        → Default → Continue
```

### Key Concepts of the Handoff Pattern

1. **Trigger Conditions**
   - Complexity-based: Simple requests vs complex requests
   - Expertise-based: General knowledge vs specialized knowledge
   - Service level: Basic service vs premium service

2. **Context Transfer**
   - Transfer previous conversation history
   - Extract and pass on important information
   - Specify reasons for the handoff

3. **Role Separation**
   - Initial Triage Agent
   - Specialist Agents
   - Final Coordinator Agent

### Use Cases

- **Travel Planning**: Simple travel → Custom travel → Luxury travel → VIP concierge
- **Customer Support**: General inquiry → Complex inquiry → Technical expert → Manager
- **Medical Systems**: Symptom check → General practitioner → Specialist → Surgical team
- **Financial Services**: Simple inquiry → Complex transaction → Approval required → Manager approval

In [ ]:
"""
Handoff Pattern - MAF WorkflowBuilder usage
- Define handoff nodes with the @executor decorator
- Create dynamic routing graphs with WorkflowBuilder
- Triage → Complexity evaluation → Specialist selection → Coordinator
"""

# ========================================================================
# Define Message Types (for Handoff Pattern)
# ========================================================================

@dataclass
class TravelPlanRequest:
    """Travel plan request message (including handoff)"""
    travel_request: str
    complexity_score: int = 0
    triage_analysis: Optional[str] = None
    assigned_specialist: str = "undetermined"
    specialist_response: Optional[str] = None
    final_proposal: Optional[str] = None
    status: str = "pending"

# ========================================================================
# Create Handoff Pattern Agents
# ========================================================================

def create_handoff_agents():
    """Create agents for handoff"""
    
    if not agent_client:
        raise ValueError("❌ MAF Agent client has not been initialized.")
    
    print("\n" + "="*70)
    print("🤖 Creating Handoff Pattern Agents.")
    print("="*70)
    
    # Agent 1: Triage Agent (Classification and Complexity Evaluation)
    triage_agent = agent_client.create_agent(
        name="TriageAgent",
        instructions=(
            "You are a travel request classification expert.\n\n"
            "Role:\n"
            "- Accurately evaluate the complexity of a travel request (score: 0-100)\n"
            "- Determine the required level of expertise\n"
            "- Recommend the suitable type of specialist\n"
            "- Analyze keywords and characteristics\n\n"
            "Evaluation criteria:\n"
            "- Score 0-30: Simple trip (major cities, short duration, general sightseeing)\n"
            "- Score 31-70: Customized trip (theme-based, multiple cities, specific interests)\n"
            "- Score 71-100: Luxury/Complex trip (private tours, high-end accommodations, unique experiences)\n\n"
            "Style: Precise and objective analysis"
        )
    )
    
    # Agent 2: General Travel Advisor
    general_advisor = agent_client.create_agent(
        name="GeneralAdvisor",
        instructions=(
            "You are a friendly general travel advisor.\n\n"
            "Role:\n"
            "- Provide simple and practical travel plans\n"
            "- Recommend 3-5 main attractions\n"
            "- Mid-range hotels and public transport details\n"
            "- Rough budget guide\n"
            "- Suggest popular restaurants and cafes\n\n"
            "Style: Simple and approachable explanations"
        )
    )
    
    # Agent 3: Specialist Travel Planner
    specialist_planner = agent_client.create_agent(
        name="SpecialistPlanner",
        instructions=(
            "You are a specialist travel planner.\n\n"
            "Role:\n"
            "- Tailored travel plans reflecting client interests\n"
            "- Proposed themed itineraries and unique experiences\n"
            "- Opportunities for local cultural immersion\n"
            "- Hidden gems and local hotspots\n"
            "- Boutique hotels and Michelin star restaurants\n"
            "- Optimized transportation and detailed budgeting\n\n"
            "Style: Professional and creative planning"
        )
    )
    
    # Agent 4: Luxury Travel Concierge
    luxury_concierge = agent_client.create_agent(
        name="LuxuryConcierge",
        instructions=(
            "You are a luxury travel concierge.\n\n"
            "Role:\n"
            "- Provide top-class service and exclusive experiences\n"
            "- VIP-only experiences and private tours\n"
            "- 5-star hotels or private villas\n"
            "- Reservations at Michelin 3-star restaurants\n"
            "- Dedicated transport and guide services\n"
            "- Premium spa & wellness\n"
            "- 24-hour concierge services\n\n"
            "Style: Elegant and sophisticated suggestions"
        )
    )
    
    # Agent 5: Coordinator (Finalization)
    coordinator = agent_client.create_agent(
        name="Coordinator",
        instructions=(
            "You are a travel coordinator.\n\n"
            "Role:\n"
            "- Consolidate specialist plans into high-quality proposal\n"
            "- Ensure completeness of travel plans\n"
            "- Provide a clear itinerary format\n"
            "- Budget summary and reservation guidelines\n"
            "- Information on next steps and contact details\n"
            "- Present additional customization options\n\n"
            "Style: Structured and polished documentation"
        )
    )
    
    print(f"✅ Agent 1: {triage_agent.name} (Triage Agent) successfully created")
    print(f"✅ Agent 2: {general_advisor.name} (General Advisor) successfully created")
    print(f"✅ Agent 3: {specialist_planner.name} (Specialist Planner) successfully created")
    print(f"✅ Agent 4: {luxury_concierge.name} (Luxury Concierge) successfully created")
    print(f"✅ Agent 5: {coordinator.name} (Coordinator) successfully created")
    print("="*70 + "\n")
    
    return triage_agent, general_advisor, specialist_planner, luxury_concierge, coordinator

# Create Agents for Handoff
triage_agent, general_advisor, specialist_planner, luxury_concierge, coordinator = create_handoff_agents()

# ========================================================================
# Step 1: Define Executor Nodes (Handoff Pattern)
# ========================================================================

@executor(id="triage_agent")
async def triage_agent_node(msg: TravelPlanRequest, ctx: WorkflowContext[TravelPlanRequest]) -> None:
    """Step 1: Analyze travel requests and assess complexity"""
    print(f"\n🔍 Step 1: Triage Agent - Analyzing travel request.")
    print(f"📩 Request: {msg.travel_request[:100]}.")
    
    query = f"""Analyze the following travel request and assess its complexity:

Travel Request: {msg.travel_request}

Evaluate the following aspects:
1. Complexity score (0-100)
2. Category (Simple/Customized/Luxury)
3. Reason for assessment
4. Key keywords
5. Recommended specialist type

Evaluation criteria:
- 0-30: Simple trip (major cities, short duration, general sightseeing)
- 31-70: Customized trip (theme-based, multiple cities, specific interests)
- 71-100: Luxury/Complex trip (private tours, high-end accommodations, unique experiences)

"""
    
    # Execute Triage Agent
    thread = triage_agent.get_new_thread()
    result = await triage_agent.run(query, thread=thread)
    response = result.text if hasattr(result, 'text') else str(result)
    
    msg.triage_analysis = response
    
    # Extract complexity score (based on keywords)
    if any(word in response for word in ['simple', 'short', 'weekend', 'general']):
        msg.complexity_score = 20
        msg.assigned_specialist = "General Advisor"
        target_specialist = "general_advisor"
    elif any(word in response for word in ['luxury', 'complex', 'private', 'special', 'high-end', 'VIP']):
        msg.complexity_score = 85
        msg.assigned_specialist = "Luxury Concierge"
        target_specialist = "luxury_concierge"
    else:
        msg.complexity_score = 55
        msg.assigned_specialist = "Specialist Planner"
        target_specialist = "specialist_planner"
    
    msg.status = "analyzed"
    
    print(f"✅ Analysis completed")
    print(f"🎯 Complexity Score: {msg.complexity_score}/100")
    print(f"🎯 Assigned Specialist: {msg.assigned_specialist}")
    print(f"Analysis preview:\n{response[:300]}.\n")
    
    # Handoff to the appropriate specialist
    await ctx.send_message(msg, target_id=target_specialist)


@executor(id="general_advisor")
async def general_advisor_node(msg: TravelPlanRequest, ctx: WorkflowContext[TravelPlanRequest]) -> None:
    """Step 2a: General travel advisory"""
    print(f"\n✈️  Step 2a: General Advisor - Preparing basic travel plan.")
    
    query = f"""Provide a friendly and clear basic travel plan for the following request:

Request: {msg.travel_request}

Analysis result: {msg.triage_analysis[:200]}"""
    
    # Execute General Advisor Agent
    thread = general_advisor.get_new_thread()
    result = await general_advisor.run(query, thread=thread)
    response = result.text if hasattr(result, 'text') else str(result)
    
    msg.specialist_response = response
    msg.status = "specialist_completed"
    
    print(f"✅ Basic travel plan completed")
    print(f"Plan preview:\n{response[:300]}.\n")
    
    # Handoff to Coordinator
    await ctx.send_message(msg, target_id="coordinator")


@executor(id="specialist_planner")
async def specialist_planner_node(msg: TravelPlanRequest, ctx: WorkflowContext[TravelPlanRequest]) -> None:
    """Step 2b: Specialist travel planning"""
    print(f"\n🎨 Step 2b: Specialist Planner - Preparing tailored travel plan.")
    
    query = f"""Create a detailed tailored travel plan for the following request:

Request: {msg.travel_request}

Analysis result: {msg.triage_analysis[:200]}"""
    
    # Execute Specialist Planner Agent
    thread = specialist_planner.get_new_thread()
    result = await specialist_planner.run(query, thread=thread)
    response = result.text if hasattr(result, 'text') else str(result)
    
    msg.specialist_response = response
    msg.status = "specialist_completed"
    
    print(f"✅ Tailored travel plan completed")
    print(f"Plan preview:\n{response[:300]}.\n")
    
    # Handoff to Coordinator
    await ctx.send_message(msg, target_id="coordinator")


@executor(id="luxury_concierge")
async def luxury_concierge_node(msg: TravelPlanRequest, ctx: WorkflowContext[TravelPlanRequest]) -> None:
    """Step 2c: Luxury travel concierge"""
    print(f"\n💎 Step 2c: Luxury Concierge - Preparing luxury travel plan.")
    
    query = f"""Create the ultimate luxury travel plan for the following request:

Request: {msg.travel_request}

Analysis result: {msg.triage_analysis[:200]}"""
    
    # Execute Luxury Concierge Agent
    thread = luxury_concierge.get_new_thread()
    result = await luxury_concierge.run(query, thread=thread)
    response = result.text if hasattr(result, 'text') else str(result)
    
    msg.specialist_response = response
    msg.status = "specialist_completed"
    
    print(f"✅ Luxury travel plan completed")
    print(f"Plan preview:\n{response[:300]}.\n")
    
    # Handoff to Coordinator
    await ctx.send_message(msg, target_id="coordinator")


@executor(id="coordinator")
async def coordinator_node(msg: TravelPlanRequest, ctx: WorkflowContext[TravelPlanRequest]) -> None:
    """Step 3: Finalizing the travel proposal"""
    print(f"\n📝 Step 3: Coordinator - Compiling final travel proposal.")
    
    query = f"""Review the following specialist travel plan and create the final customer proposal:

Original Request: {msg.travel_request}
Complexity: {msg.complexity_score}/100
Assigned Specialist: {msg.assigned_specialist}

Specialist Plan:
{msg.specialist_response[:800]}.

Final Proposal Guidelines:
1. Confirm completeness of the travel plan
2. Check for reservation availability and feasibility
3. Compile into a clear itinerary format
4. Add budget summary
5. Provide reservation process and next steps
6. Suggest additional customization options
7"""
    
    # Execute Coordinator Agent
    thread = coordinator.get_new_thread()
    result = await coordinator.run(query, thread=thread)
    response = result.text if hasattr(result, 'text') else str(result)
    
    msg.final_proposal = response
    msg.status = "completed"
    
    print(f"✅ Final proposal completed")
    
    await ctx.yield_output(msg)


# ========================================================================
# Step 2: Build Handoff Workflow Graph with WorkflowBuilder
# ========================================================================

handoff_workflow = (
    WorkflowBuilder()
    # Start with Triage Agent
    .set_start_executor(triage_agent_node)
    # Triage → 3 Specialists (Dynamic Routing)
    .add_edge(triage_agent_node, general_advisor_node)
    .add_edge(triage_agent_node, specialist_planner_node)
    .add_edge(triage_agent_node, luxury_concierge_node)
    # Each Specialist → Coordinator
    .add_edge(general_advisor_node, coordinator_node)
    .add_edge(specialist_planner_node, coordinator_node)
    .add_edge(luxury_concierge_node, coordinator_node)
    .build()
)

print("\n" + "="*70)
print("✅ Handoff Workflow Graph successfully built")
print("="*70)
print("📊 Workflow Structure (Handoff Pattern):")
print("   triage_agent_node (Complexity Evaluation)")
print("         ↓")
print("   [Dynamic Routing]")
print("    ├─→ general_advisor_node (Simple Travel)")
print("    ├─→ specialist_planner_node (Customized Travel)")
print("    └─→ luxury_concierge_node (Luxury Travel)")
print("         ↓")
print("   coordinator_node (Finalization)")
print("="*70 + "\n")


# ========================================================================
# Step 3: Execute Workflow
# ========================================================================

async def run_handoff_workflow(travel_request: str):
    """Function to execute the Handoff Workflow"""
    print("\n" + "="*70)
    print("🚀 Starting Handoff Workflow Execution (MAF WorkflowBuilder)")
    print("="*70)
    print(f"📩 Travel Request: {travel_request[:100]}.")
    print("="*70)

    # Create input message
    plan_request = TravelPlanRequest(
        travel_request=travel_request
    )

    # Execute Workflow (using run_stream - async generator)
    outputs = []
    async for event in handoff_workflow.run_stream(plan_request):
        # Extract output from the event
        if hasattr(event, 'output') and event.output is not None:
            outputs.append(event.output)
            print(f"📤 Event received: {type(event.output).__name__}")
        elif hasattr(event, 'data') and event.data is not None:
            outputs.append(event.data)
            print(f"📤 Event received: {type(event.data).__name__}")

    # Extract final result
    if outputs:
        final_result = outputs[-1]
    else:
        raise ValueError("❌ No output received from the workflow.")

    print("\n" + "="*70)
    print("📊 Handoff Pattern Result (MAF WorkflowBuilder)")
    print("="*70)
    print(f"\n📝 Travel Request: {final_result.travel_request[:100]}.")
    print(f"✅ Complexity Score: {final_result.complexity_score}/100")
    print(f"✅ Assigned Specialist: {final_result.assigned_specialist}")
    print(f"✅ Final Status: {final_result.status}")
    
    print(f"\n{'='*70}")
    print(f"🔍 Complexity Analysis:")
    print(f"{'='*70}")
    print(f"{final_result.triage_analysis}\n")
    
    print(f"\n{'='*70}")
    print(f"💬 Specialist Response:")
    print(f"{'='*70}")
    print(f"{final_result.specialist_response}\n")
    
    print(f"\n{'='*70}")
    print(f"✨ Final Travel Proposal:")
    print(f"{'='*70}")
    print(f"{final_result.final_proposal}")
    
    print(f"\n{'='*70}")
    
    return final_result

# Execute Handoff Workflow - Testing various complexities
print("\n\n🧪 Scenario 1: Simple Travel - Handoff to General Travel Advisor Agent")
result1 = await run_handoff_workflow(
    "Please plan a weekend trip to Busan for 2 nights and 3 days. Recommend major attractions and good restaurants."
)

# Scenario 2: Customized Travel Plan (Specialist Travel Planner)
print("\n\n🧪 Scenario 2: Customized Travel - Handoff to Specialist Travel Planner")
result2 = await run_handoff_workflow(
    "I want to plan a 5-day healing trip in Kyoto, Japan, centered around traditional culture and Zen meditation. I would like temple stays, tea ceremony experiences, and garden tours for in-depth cultural experiences."
)

# Scenario 3: Luxury Travel (Luxury Travel Concierge)
print("\n\n🧪 Scenario 3: Luxury Travel - Handoff to Luxury Travel Concierge")
result3 = await run_handoff_workflow(
    "I need a 2-week luxury tour in Europe for a family of four. We want to visit Paris, Milan, and Santorini, including Michelin 3-star restaurants, 5-star hotels, private guided tours, and special VIP experiences. No budget limit, seeking the best experiences."
)


### Handoff Pattern Analysis and Summary

#### 📊 Advantages of the Handoff Pattern

1. **Utilization of Adaptive Expertise**
   - Assign an appropriate level of expert based on travel complexity
   - Efficient use of resources (avoids excessive expert involvement)
   - Allows each agent to focus on their area of expertise

2. **Scalable Structure**
   - Easy to add new expert agents
   - Flexible routing by adjusting complexity criteria
   - Expands into various domains seamlessly

3. **Quality Assurance**
   - Triage Agent performs accurate classification
   - Each expert provides optimized travel solutions
   - Coordinator ensures final quality validation

4. **Tracking and Monitoring**
   - Analyze pattern via handoff history
   - Identify and improve bottlenecks
   - Measure agent performance individually

#### 🎯 Practical Application Examples

**Travel Planning Service**
```
Level 1 → Simple travel (general consultant - package tours)
Level 2 → Custom travel (specialized planner - theme travel)
Level 3 → Luxury travel (concierge - VIP experiences)
Level 4 → Ultra-premium (personalized manager - fully tailored)
```

**Customer Support Center**
```
Level 1 → General FAQ (automated response)
Level 2 → Technical support (specialized consultant)
Level 3 → Advanced engineer (complex problems)
Level 4 → Manager approval (policy changes required)
```

**Medical System**
```
Symptom check → Nurse (general consultation)
               → General physician (diagnosis required)
               → Specialist (special treatment)
               → Surgery team (surgery required)
```

#### ⚙️ Implementation Considerations

1. **Clear Complexity Criteria**
   - Quantitative criteria (keywords, request length, budget, etc.)
   - Qualitative criteria (domain-specific rules)
   - Potential use of ML models based on training data

2. **Optimized Context Transmission**
   - Share only essential information (avoid excessive context)
   - Use structured formats (JSON, etc.)
   - Specify reasons behind the handoff

3. **Escalation Policies**
   - Define automatic escalation conditions
   - Support manual escalation requests
   - Track escalation history

4. **Performance Optimization**
   - Use caching to avoid redundant analyses
   - Reduce wait times with asynchronous processing
   - Identify segments for parallel processing

#### 🔄 Combination with Other Patterns

**Handoff + Sequential**
```python
# Workflow that sequentially involves multiple experts
triage → travel_planner → budget_optimizer → booking_agent
```

**Handoff + Parallel**
```python
# Obtain advice from multiple experts simultaneously, then integrate
triage → [flight_expert, hotel_expert, activity_expert] → coordinator
```

**Handoff + Loop**
```python
# Iterative improvement until satisfaction is achieved
triage → specialist → customer_review → (unsatisfied) → specialist (retry)
                                        → (satisfied) → complete
```

## 📊 Comprehensive Workflow Pattern Comparison

### Summary of Features by Pattern (Lab Order)

| Lab | Pattern | Execution Method | Time Complexity | Primary Use Cases | Key Advantages | Considerations |
|------|--------|------------------|------------------|-------------------|----------------|----------------|
| **Lab 2** | **Sequential** | Sequential Execution | T1 + T2 + T3 | Trip Itinerary Writing → Local Expert Review → Final Touch | Clear Workflow, Simple Implementation | Slow Execution Speed |
| **Lab 3** | **Concurrent** | Parallel Execution | max(T1, T2, T3) | Multi-City Analysis, Multidimensional Review | Fast Processing, Diverse Perspectives | Requires Synchronization Management |
| **Lab 4** | **Conditional** | Conditional Branching | Classifier + Expert | Routing Based on Travel Style | Personalized Response, Efficient Processing | Classification Accuracy is Critical |
| **Lab 5** | **Loop** | Iterative Updates | N × T | Iterative Improvement of Travel Plans, Quality Enhancement | Incremental Improvement, Feedback Integration | Must Prevent Infinite Loops |
| **Lab 6** | **Error Handling** | Retry + Fallback | Variable | Alternative Suggestions for Booking Failures | Stability, Resilience, Automatic Recovery | Additional Overhead |
| **Lab 7** | **Handoff** | Dynamic Routing | Triage + Specialist | Assigning Experts Based on Complexity | Adaptive Expertise, Resource Optimization | Clear Complexity Metrics Required |

### Decision Tree for Choosing a Pattern

```
Analysis of Travel Planning Tasks:
│
├─ Are the sequential steps clear?
│  ├─ Yes → Sequential Pattern (Lab 2)
│  │         (e.g., Itinerary Writing → Verification → Booking)
│  └─ No → Proceed to the next step
│
├─ Are multiple independent perspectives needed?
│  ├─ Yes → Concurrent Pattern (Lab 3)
│  │         (e.g., Simultaneous search for flights, accommodations, activities)
│  └─ No → Proceed to the next step
│
├─ Are different experts needed based on input conditions?
│  ├─ Yes → Conditional Pattern (Lab 4)
│  │         (e.g., Selecting experts based on travel style)
│  └─ No → Proceed to the next step
│
├─ Is iterative improvement required?
│  ├─ Yes → Loop Pattern (Lab 5)
│  │         (e.g., Refining until customer satisfaction goals are met)
│  └─ No → Proceed to the next step
│
├─ Is handling external service failures necessary?
│  ├─ Yes → Error Handling Pattern (Lab 6)
│  │         (e.g., Automatic recovery for booking failures)
│  └─ No → Proceed to the next step
│
└─ Is differentiation by complexity essential?
   └─ Yes → Handoff Pattern (Lab 7)
             (e.g., Simple trips vs. Luxury trips)
```

### Practical Combination Patterns

In real-world projects, multiple patterns are often combined:

**Combination 1: Sequential + Error Handling (Lab 2 + Lab 6)**
```
Itinerary Writing → Booking Attempt → (On Failure) Alternative Suggestion → Rebooking → Confirm
```
- Use Case: Reliable Travel Booking Systems

**Combination 2: Concurrent + Conditional (Lab 3 + Lab 4)**
```
[Culture Expert, Food Expert, Activity Expert] → Style-Based Filtering → Integration
```
- Use Case: Personalized Travel Recommendation Systems

**Combination 3: Handoff + Loop (Lab 7 + Lab 5)**
```
Complexity Evaluation → Expert Assignment → Plan Creation → Feedback → (Improvement) Reiteration → Completion
```
- Use Case: Premium Travel Consulting

**Combination 4: Handoff + Sequential (Lab 7 + Lab 2)**
```
Complexity Evaluation → Expert Selection → Itinerary Writing → Verification → Confirmation
```
- Use Case: General Travel Planning Services

### Key Concept Summary by Lab

**Lab 2 - Sequential Pattern**
- Concept: Three agents execute tasks in sequence
- Structure: planner → reviewer → finalizer
- Key Learning Points: Connect sequentially with `add_edge()`, accumulate results at each step

**Lab 3 - Concurrent Pattern**
- Concept: Multiple agents perform tasks simultaneously and results are integrated
- Structure: broadcast → [culture, food, practical] → aggregator
- Key Learning Points: Use `broadcast_start_node` for fan-out, execute multiple paths in parallel

**Lab 4 - Conditional Pattern**
- Concept: Select different experts based on input
- Structure: classifier → [culture/activity/relaxation/gourmet expert]
- Key Learning Points: Use dynamic `target_id` for conditional routing

**Lab 5 - Loop Pattern**
- Concept: Iteratively improve until conditions are met
- Structure: generator ↔ analyzer (up to 3 cycles) → optimizer
- Key Learning Points: Track `iteration_count`, manage conditional loops and termination

**Lab 6 - Error Handling Pattern**
- Concept: Detect failure → Analyze → Suggest Alternatives → Recovery
- Structure: booking → error_analyzer → fallback_planner → recovery
- Key Learning Points: Detect errors using keywords, generate automatic alternatives

**Lab 7 - Handoff Pattern**
- Concept: Delegate responsibility to an appropriate expert after complexity evaluation
- Structure: triage → [general/specialist/luxury advisor] → coordinator
- Key Learning Points: Dynamic routing, assigning experts based on complexity

---

## 🎯 Azure AI Foundry Agent Best Practices

### 1. Principles for Writing Prompts

#### ✅ Good Prompt Example
```python
prompt = """
Task: Create a 5-day travel itinerary for Paris.

Requirements:
- Duration: November 15, 2025 ~ November 19, 2025
- Interests: Museums, café culture, local cuisine
- Budget: Mid-to-high range

Output format:
- Detailed daily schedule (hourly)
- Recommendations for 3 restaurants
- Transportation information
- Estimated costs

Write in Korean.
"""
```

#### ❌ Poor Prompt Example
```python
prompt = "Make me a travel plan for Paris"  # Too vague
```

### 2. Define Agent Roles

Assign **clear roles and expertise** to each agent:

```python
# Travel Planner Agent
instructions = """
You are a professional travel planner.

Expertise:
- Schedule optimization
- Understanding local culture
- Budget management
- Hidden gem recommendations

Workflow:
1. Accurately understand customer requirements
2. Create feasible itineraries
3. Provide alternative options
4. Offer explanations with clear reasoning
"""
```

### 3. Error Handling Strategy

```python
# ✅ Systematic error handling
try:
    result = await call_foundry_agent(query, instructions)
    
    # Validate the result
    if not result or len(result) < 50:
        raise ValueError("The response is too short")
        
except Exception as e:
    logger.error(f"Agent call failed: {e}")
    
    # Fallback strategy
    fallback_result = await call_foundry_agent(
        simplified_query,
        fallback_instructions
    )
```

### 4. Performance Optimization

#### Utilizing Parallel Processing
```python
# ✅ Independent tasks processed concurrently
results = await asyncio.gather(
    call_foundry_agent(query1, instructions1),
    call_foundry_agent(query2, instructions2),
    call_foundry_agent(query3, instructions3)
)
```

#### Token Usage Optimization
- ✅ Include only essential information in the context
- ✅ Summarize past conversations before passing them along
- ✅ Remove unnecessary repetitive explanations

### 5. Practical Checklist

**Before Agent Invocation:**
- [ ] Is the prompt clear and specific?
- [ ] Is the output format defined?
- [ ] Is the agent role clearly outlined?
- [ ] Is the expected token usage appropriate?

**After Agent Invocation:**
- [ ] Does the response meet the requirements?
- [ ] Is error handling implemented?
- [ ] Was the execution time within permissible limits?
- [ ] Are logs appropriately recorded?

### 6. Key Precautions

⚠️ **Prevent Infinite Loops**
```python
max_iterations = 10  # Set a maximum iteration count
for i in range(max_iterations):
    result = await process()
    if condition_met:
        break
```

⚠️ **Set a Timeout**
```python
import asyncio

try:
    result = await asyncio.wait_for(
        call_foundry_agent(query, instructions),
        timeout=30.0  # 30-second timeout
    )
except asyncio.TimeoutError:
    logger.error("Request timeout")
```

⚠️ **Resource Cleanup**
```python
# Ensure all created agents and threads are cleaned up
try:
    # Perform tasks
    result = await process()
finally:
    # Clean up resources
    if agent_id:
        foundry_client.agents.delete_agent(agent_id)
    if thread_id:
        foundry_client.agents.threads.delete(thread_id)
```

## 📍 Next Steps

You have completed learning the MAF workflow pattern! Now proceed to the next notebooks in order:

1. **Notebook 06**: MAF Dev UI Lab (`06_maf_dev_ui.ipynb`)
2. **Notebook 07**: Agent Evaluation (`07_evaluate_agents.ipynb`)

Best of luck on your AI agent development journey! 💪